# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.15 热门AI应用实战一：将谷歌搜索接入GPT

- 基于大模型的AI应用开发现状

&emsp;&emsp;在掌握了基于Function calling的AI应用开发流程之后，接下来我们将进一步介绍一些热门领域的AI应用开发案例。截至目前，最为热门的围绕大模型进行AI应用开发的方向主要有两个，其一是围绕特定领域的知识库训练问答机器人，也就是所谓的本地知识库问答；其二则是通过“训练”大模型使其能够完成某些特定领域的某些工作，例如编写小说、解读论文、自然语言编程等，在这些应用中，和数据技术人最为息息相关的一类应用就是特定领域的代码解释器——即一整套能够全自动执行复杂代码流程的AI应用。当然，围绕这两个大模型应用方向，其实是有多种类型不同的具体需求已经对应的解决方案的，并且哪怕是同样一个本地知识库问答系统或者代码解释器，也肯定存在非常多种不同类型的解决方案，并且截至目前，并没有哪一种技术方案是一定完成AI应用开发最好的解决方案。事实上，围绕这些需求进行AI应用开发，正逐渐变为类似机器学习算法建模一般的存在——有非常多种方法可以实现（建模），但具体实现效果（模型准确率）会非常依赖大模型工程师的技术水平。

&emsp;&emsp;因此，在本次课程中，我们将挑选其中的大模型本地知识库问答以及特定领域的代码解释器作为重点探讨的项目内容，并且正如此前所说，这些AI落地应用往往会有多种不同类型的形式，同时也会涉及非常多种不同的技术方案，因此我们也会伴随着课程推进、伴随着我们所介绍的技术手段越来越多样，围绕这些项目提出各类不同的解决策略。

&emsp;&emsp;而在接下来的三个小节中，我们就将首先围绕这些方向，介绍一些入门级需求场景及解决方案：在Ch.15中，我们将重点介绍如何将谷歌搜索接入Chat模型中，从而一定程度解决大语言模型本身知识库时效性和专业性不足的问题，这相当于是定制化知识库问答的一种实现形式；而在Ch.16中，我们将尝试将MySQL接入Chat模型，从而实现某些具体需求场景下的定制化SQL代码解释器；而在Ch.17中，我们则将进一步尝试借助Function calling功能来解决本地知识库内容长度超出模型最大上下文限制的问题，尝试围绕课程中的某部分课件，来训练定制化问答机器人。而从Ch.18开始往后，我们则将正式进入到Embedding和微调这一全新的技术领域中，而这些技术其实也正是解决长文本问答机器人的非常核心的技术，因此在介绍完这些技术的技术原理之后，其非常重要的实践环节就是再次尝试将这些技术应用到AI应用开发中去。

&emsp;&emsp;本节我们就将首先介绍如何借助Function calling将谷歌搜索接入Chat模型，以一定程度解决大语言模型知识库时效性不足的问题。当然，这也属于定制化知识库问答系统这类应用需求的某个具体的表现形式，因此在课程开始之前，我们需要先介绍下导致大语言模型知识库的“时效性”与“专业性”缺陷的根本原因，以及目前可以解决该问题的技术手段。

- 大语言模型知识库缺陷

&emsp;&emsp;尽管GPT大模型所具备的知识浩如烟海，远超个人能掌握的知识范畴，但受限于模型本身的训练机制，大模型原始模型（非微调模型）背后的知识库只能够在预训练时才会被更新，换而言之也就是大模型的知识库是由预训练阶段输入的数据信息所决定的。而截至目前，大模型的预训练仍然是非常复杂且极为“烧钱”的，目前业内普遍认为一次大模型的预训练成本约在200万美元到1200万美元之间，且背后还需要大量的算力支持。考虑到预训练还附带实验成本和极高的技术门槛，大模型预训练对于普通企业来说几乎是不可能做到的事情，而业内发布的大语言模型也都是阶段性推出新版本模型（例如GPT3、GPT4、LAMMA、LAMMA2），且往往模型版本更新时才会更新知识库，因此一般来说大语言模型知识库的时效性都落后于当前时间1-3年。例如GPT3、GPT4模型的知识库截止2021年9月，而LAMMA2模型的知识库则是截止2022年9月。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/c2ec69756a3e0c7d9b9592f2c87fb23.png" alt="c2ec69756a3e0c7d9b9592f2c87fb23" style="zoom:33%;" />

> 相比GPT-3，GPT-4模型只调整的模型结构和训练方法，并没有调整背后的知识库。

&emsp;&emsp;同时，对于某些高度专业的知识，例如一些特定领域的特征工程方法或者超参数优化策略，若这些高度专业的知识在预训练数据集中占比很小（甚至根本没有），那么训练得到的模型是无法对这些问题做出准确回答的，这也是当前人们将大预言模型应用至特定领域所遇到的最核心的问题。

&emsp;&emsp;而无论是大模型的时效性息缺失还是专业领域知识匮乏，其背后的根本原因都是大模型知识库存在一定的“缺陷”——通用大模型的知识库只能按阶段更新，且越是专业领域的知识占比越少。因此，如何解决这些问题，就成了当前大模型应用的最热门的方向——如何引导大模型围绕本地知识库进行问答。当然，这里的本地知识库有可能是更新的知识内容，也可能是特定领域高度专业性知识。

- 定制化知识库问答机器人基本技术方向

&emsp;&emsp;而截止目前，总的来看，大模型本地知识库问答策略有两个方向，其一是借助提示工程，使得模型在每次回答之前都能接收到与这个问题最相关的一些文案，从而让模型能够更好的完成回答。例如我们在Ch.7中曾介绍将课件中关于Chat模型的使用方法作为system message输入到模型中，从而使得模型能够回答原本超出其知识库的Chat模型使用方法的相关问题。不难发现，这种策略非常高效且易于执行，但问题在于受限于模型可接收的最大上下文长度，我们无法无休止的对其输入背景信息，而一旦信息量超过最大上下文限制，则需要节选我们认为最关键的信息对其进行输入，来引导模型更好的进行回答。当然这一过程也是可以通过技术手段来完成的，例如我们可以灵活对长文本进行切分，然后让模型自行总结每一段的核心内容（并作为函数描述），再将其封装为一个个外部函数，并在提问时开启Chat模型的Function calling功能，从而使得模型当面临自己不知道的问题时，能够自动检索每个文段的核心内容并选择与之最匹配的文段进行读取。当然，这一匹配的过程也可以通过Embedding方法，先对问题和相关文案进行编码再通过匹配关系来自动寻找和答案最相关的文段进行输入。

&emsp;&emsp;而无论是借助Function calling还是Embedding，借助提示工程来完成本地知识库问答核心都是借助大语言模型的隐藏状态进行信息的临时存储，而要想要让模型永久的“记住”某些信息，则需要通过修改模型参数来完成。正如我们在Part 5部分中所介绍的那样，只有当模型参数发生变化，模型对信息处理的流程才会发生变化，围绕某些问题的回答才会发生根本性的改变，模型才相当于是“记住”了某些信息。而除了预训练之外，能够修改模型参数的方法就只剩下微调这一种方法。因此这也就是大模型本地知识库问答的第二条技术策略——通过微调的方法，让模型永久“记住”本地知识库信息。

&emsp;&emsp;当然，“永久”记住某些信息确实看起来非常诱人，但实际上大模型微调却并不是那么简单就能实现，微调过程不仅涉及较高的技术门槛和算力门槛，而且稍有不慎甚至会引发大模型的灾难性遗忘——即新的知识没学会、还会导致旧的知识被忘记。因此，哪怕是微调技术已经快速发展了近5年时间，也诞生了一系列有监督微调和基于强化学习的微调策略，但要微调得到一个超越提示工程水平的结果，仍然是相当困难的事情。

&emsp;&emsp;在接下来的课程中，我们将逐步尝试上述各类方法以解决大模型本地知识库时效性和专业性不足的问题。从技术的角度来说，本节我们将首先尝试使用Function calling技术方法来解决大语言模型知识时效性和专业性不足的问题——即尝试将谷歌搜索接入Chat模型，通过为模型实时提供一些超过原本知识库的最新知识，来更好的完成问答任务。

In [1]:
import os
import openai
import glob
import shutil
openai.api_key = os.getenv("OPENAI_API_KEY")

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re
import random
import string

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import base64
import email
from email import policy
from email.parser import BytesParser
from email.mime.text import MIMEText

from bs4 import BeautifulSoup
import dateutil.parser as parser
import tiktoken
from lxml import etree

import sys
sys.path.insert(0, '.\\functions\\untested functions')
sys.path.insert(0, '.\\functions\\tested functions')
from gptLearning import *

---

**<center>关于OpenAI 0822更新介绍：GPT-3.5模型开放微调接口**

&emsp;&emsp;根据OpenAI 0822的更新说明，目前已经全面开放GPT-3.5微调接口，可以借助OpenAI提供的在线微调框架（fine-tuning API）进行模型微调。这是OpenAI继0613开放16K模型以及上线Function calling功能之后的又一重大更新。在此之前，OpenAI只对Completion模型开放了微调接口，即在此之前用户若想借助OpenAI在线微调框架进行在线大模型微调，最多只能微调text-davinci-003模型。而根据此前的测试，text-davinci-003模型整体性能弱于GPT-3.5模型。这也就是说，本次更新为目前OpenAI最强的在线大模型提供了微调方法。而根据更新公告，GPT-3.5微调模型在部分领域的性能将会超过GPT-4模型。毫无疑问这是在线大模型又一次重大的性能突破。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/395a5c41e1b6ded5b293285ddf630e9.png" alt="395a5c41e1b6ded5b293285ddf630e9" style="zoom:15%;" />

&emsp;&emsp;并且，不同于开源微调框架需要进行环境安装部署、消耗本地算力以及拥有较高的代码门槛，OpenAI提供的在线微调API只需要支付一定的费用，即可高效快速的完成在线微调，且代码门槛极低，只需要四步，即可完成微调全部过程。这也极大程度降低了大模型应用探索的门槛。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/aa1d9599fd8f75114a18a30a393dcc0.png" alt="aa1d9599fd8f75114a18a30a393dcc0" style="zoom:33%;" />

> 当然，Chat模型和Completion模型微调API是一致的。

&emsp;&emsp;此外根据本次的更新公告，OpenAI计划在2023年第三季度开放GPT-3.5-16K模型和GPT-4模型的微调接口，届时在线大模型即将迎来进一步的性能提升，并且作为目前通用能力最强模型之一——GPT-4模型的微调结果也十分令人期待。我们团队也在第一时间进行了GPT大模型微调实测，经验证，GPT-3.5模型在经过微调之后围绕本地知识库问答性能有了明显提升。关于GPT模型微调这部分内容，也将在不久的未来和大家见面。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/b5e81f6e836cd2ef07b5fb994652e39.png" alt="b5e81f6e836cd2ef07b5fb994652e39" style="zoom:33%;" />

---

### 一、搜索问答机器人项目思路与可行性验证

- 昙花一现的Browsing with Bing插件

&emsp;&emsp;当然，哪怕不从技术发展大框架进行分析，详细每一位使用过ChatGPT的用户都能深刻的感受到将搜索引擎接入大模型的实际价值，并且就在今年5月，OpenAI曾在ChatGPT上推出Browsing with Bing插件，该插件能够让ChatGPT遇到知识库无法解决的问题时进行Bing搜索，并根据搜索得到的答案来进行回答：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/4ec6436dd9ac36cb21868039b2d100c.png" alt="4ec6436dd9ac36cb21868039b2d100c" style="zoom:33%;" />

不得不说，该功能的出现极大程度提升了ChatGPT的实用性。但遗憾的是，由于版权因素即一些潜在的商业竞争关系，OpenAI于7月3号正式关停了该插件，并且暂时没有再次上线的计划。

&emsp;&emsp;尽管OpenAI禁止了Browsing with Bing插件使用，但将用户的某些超出大模型知识范围的问题先转化为搜索，再将搜索结果中匹配的内容输入给模型，再让模型根据这些内容进行回答，确实是极具价值潜力的AI应用方向。尽管ChatGPT已经不支持相关应用，但我们仍然可以使用Chat模型的Function calling功能+谷歌搜索API来实现类似的功能。

- 谷歌搜索API入门介绍

&emsp;&emsp;和此前介绍过的其他应用的API类似，谷歌搜索也有API可以调用，也就是说，我们完全可以在本地代码环境中通过调用谷歌搜索API来完成具体问题的搜索。需要注意的是，目前为止，目前全球各大搜索引擎厂商中，只有谷歌开放了其API以供开发者使用，当然谷歌开放的谷歌搜索API并不是真正意义上调用Google.com搜索，而是让用户允许自行申请并创建一个谷歌搜索实例用于嵌入到自己的产品中，这个谷歌搜索实例和Google.com性能类似，但功能上存在一定的区别，比如用户自行申请的谷歌搜索实例在默认情况下不会存在商业推广广告，并且会根据申请用户的所在地区进行搜索结果的调整等。并且，为了能够更好的让用户使用谷歌搜索的某个实例，谷歌为每个实例提供了非常多可定制化的搜索选项，同时在调用这个实例方面，不仅可以使用API进行调用，谷歌更是为每个实例单独提供了域名和菜单式设置方法（可以通过某个网页完成设置），因此谷歌搜索API也被称为“可编程编程引擎（Programmable Search Engine）”，而每个可编程引擎的背后的API，则被称为Custom Search API，该API可以通过谷歌云进行申请使用。

> 从普通开发者角度来说，谷歌开放搜索API可以说是极大程度惠及了非常多开发项目，使得其能够更加高效顺利的完成搜索功能的嵌套。但如果是从商业竞争角度来说，谷歌搜索开放API的意图其实在于锁死市场上其他搜索引擎发展——既然随时随地可以调用强大的谷歌搜索，重复开发搜索引擎便毫无意义。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/45a0b65ef70102fc0507c9faeb2fea6.png" alt="45a0b65ef70102fc0507c9faeb2fea6" style="zoom:33%;" />

- 借助爬虫爬取相关网站内容

&emsp;&emsp;当然，这里还有一点需要注意的是，搜索引擎API只能返回与搜索关键词最为匹配的网站连接，而要获取这些连接内容，则需要更进一步使用爬虫对这些网站内容进行爬取，因此本小节也将简单介绍下特定网站爬虫代码的编写方法。不过需要注意的是，并不是每个网站都允许被爬取内容，在使用爬虫时，必须严格按照网站的对应规则来进行操作。

- 特定领域的搜索和问答机器人

&emsp;&emsp;不过由此也不难看出，对相关网站的信息爬取的完整性和有效性，也将直接决定Chat模型接收到的文本质量好坏。因此在将搜索引擎接入Chat模型这个项目中，爬虫技术其实也会起到非常关键的作用。不过在课程的后半部分我们会介绍，如果是主要是围绕一些特定领域的技术问题进行问答（或者当前项目只用于获取某特定类型的信息），我们或许只需要限定在某个网站内进行搜索即可（谷歌搜索支持在某网站内进行搜索，例如知乎、sklearn官网等），而特定网站内的html格式趋于一致，对应的爬虫编写难度较低，更容易获取到高质量内容信息，从而获得高质量答案。因此Chat模型+谷歌搜索API的AI应用开发策略更加适合进行特定领域的问答机器人，例如课程中就将重点介绍将搜索范围限定在知乎内，然后获得高质量大模型技术知识搜索结果和问答的策略。而就特定领域的搜索和问答机器人来说，还有一个非常重要的应用方向，那就是围绕公司章程、制度、流程等内部信息的内网进行搜索和问答。不难发现，相比之下，Browsing with Bing能够顺利获取各类型不同网站的各类信息，会更适合进行通用问题的回答。不过也正是因为Browsing with Bing能够获取全网各类信息，也导致了其快速的下架。

- Chat模型接入谷歌搜索的基本开发思路

&emsp;&emsp;基于此前课程的介绍，相信大家对基于Function calling功能来调用外部工具API实现某种AI应用应该并不感到陌生，考虑到谷歌搜索能根据关键词返回所搜结果——也就是对应结果的网站，并按照关联度强弱进行排序，我们首先不妨先进行头脑风暴，思考如何才能顺利将谷歌搜索API嵌入Chat模型中，并为其实时更新知识库。

&emsp;&emsp;首先我们知道，对于Chat模型来说，是知道自己知道或者自己不知道的（否则也就不会存在Function calling功能）。并且，其内部是具备某种机制来判断当前问题是否知道。这里我们可以通过如下方式进行测试：

In [4]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "根据用户输入的问题进行回答，如果知道问题的答案，请回答问题答案，如果不知道问题答案，请回复‘抱歉，这个问题我并不知道’"},
    {"role": "user", "content": "请问，什么是机器学习？"}
  ]
)
response.choices[0].message['content']

'机器学习是一种人工智能的分支，指的是通过从大量数据中学习和获取知识，以便让计算机能够自动从新数据中进行学习和改进，并做出预测或决策。它依赖于统计学和算法来构建模型，通过对数据的分析和运算，使计算机能够识别、理解和预测模式。通过机器学习，计算机可以自动适应不断变化的环境和任务，从而提供更准确和智能的解决方案。'

In [23]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "根据用户输入的问题进行回答，如果知道问题的答案，请回答问题答案，如果不知道问题答案，请回复‘抱歉，这个问题我并不知道’"},
    {"role": "user", "content": "请问，你知道RLHF算法么？"}
  ]
)
response.choices[0].message['content']

'抱歉，这个问题我并不知道。'

并且，当开启Function calling功能时，面对哪怕是模型知识库内已知的内容，模型仍然会优先调用外部函数进行回答：

In [29]:
def ML_answer(q='什么是机器学习'):
    """
    解释什么是机器学习，返回机器学习的定义和解释
    :param q: 询问的问题，非必要参数，字符串类型对象
    :return：返回机器学习的定义和解释
    """
    return("机器学习是一种引导工业机器进行学习的方法。")

In [30]:
ML_answer()

'机器学习是一种引导工业机器进行学习的方法。'

In [31]:
functions_list = [ML_answer]

In [32]:
functions = auto_functions(functions_list)
functions

[{'name': 'ML_answer',
  'description': '解释什么是机器学习，返回机器学习的定义和解释',
  'parameters': {'type': 'object',
   'properties': {'q': {'type': 'string', 'description': '询问的问题'}},
   'required': []}}]

In [33]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "请问，什么是机器学习？"}
        ],
        functions=functions,
        function_call="auto",  
    )
response["choices"][0]["message"]

<OpenAIObject at 0x1cfad43acf0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "ML_answer",
    "arguments": "{\n  \"q\": \"\u673a\u5668\u5b66\u4e60\u662f\u4ec0\u4e48\"\n}"
  }
}

因此，若想让谷歌搜索API的搜索起到补充知识库的作用，我们需要首先创建一层判别层，即让大模型自行判断围绕当前问题，是否要调用谷歌搜索API来回答。这个判别层可以单独采用某个大模型来进行判别，根据模型实际输出结果的知道与否，来判断是否需要调用搜索API来获取外部信息再来进行回答：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/71c2f15cbe4c470594024d8b4edb525.png" alt="71c2f15cbe4c470594024d8b4edb525" style="zoom:33%;" />

当然，还有另一种非常“巧妙”的方法，即我们可以在外部函数说明文档中强调，当且仅当模型无法回答时再调用该函数，此时由于模型本身会读取外部函数说明文档，因此模型是能够判断调用外部函数的时机的。例如我们按照如下方式创建一个外部函数：

In [34]:
def ST_answer(q='某个Chat模型无法回答的问题'):
    """
    当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 询问的问题，字符串类型对象
    :return：某问题的答案
    """
    return("接下来将调用谷歌搜索API来进行问题答案的搜索，并据此返回最终结果")

In [35]:
ST_answer()

'接下来将调用谷歌搜索API来进行问题答案的搜索，并据此返回最终结果'

In [36]:
functions_list = [ST_answer]

In [37]:
functions = auto_functions(functions_list)
functions

[{'name': 'ST_answer',
  'description': '当你无法回答某个问题时，调用该函数，能够获得答案',
  'parameters': {'type': 'object',
   'properties': {'q': {'type': 'string', 'description': '询问的问题'}},
   'required': ['q']}}]

然后测试模型能否判断调用该函数的时机：

In [64]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
            {"role": "user", "content": "请问，什么是机器学习？"}
        ],
        functions=functions,
        function_call="auto",  
    )
response["choices"][0]["message"]

<OpenAIObject at 0x1cfad4105e0> JSON: {
  "role": "assistant",
  "content": "\u673a\u5668\u5b66\u4e60\u662f\u4eba\u5de5\u667a\u80fd\u7684\u4e00\u4e2a\u5206\u652f\uff0c\u4e3b\u8981\u662f\u5173\u4e8e\u5982\u4f55\u8ba9\u7535\u8111\u7cfb\u7edf\u4ece\u7ecf\u9a8c\u5b66\u4e60\u4e2d\u83b7\u53d6\u65b0\u7684\u77e5\u8bc6\u6216\u6280\u80fd\uff0c\u589e\u5f3a\u81ea\u8eab\u7684\u6027\u80fd\u3002\u5728\u5904\u7406\u4e00\u4e9b\u8bf8\u5982\u8bc6\u522b\u3001\u9884\u6d4b\u3001\u51b3\u7b56\u7b49\u95ee\u9898\u65f6\uff0c\u4f9d\u8d56\u4e8e\u76f8\u5e94\u7684\u7b97\u6cd5\u6a21\u578b\u5bf9\u5927\u89c4\u6a21\u6570\u636e\u8fdb\u884c\u5904\u7406\uff0c\u5f97\u5230\u76f8\u5173\u6a21\u578b\uff0c\u5e76\u901a\u8fc7\u6a21\u578b\u8fdb\u884c\u9884\u6d4b\u548c\u63a8\u7406\u3002"
}

In [65]:
response["choices"][0]["message"]["content"]

'机器学习是人工智能的一个分支，主要是关于如何让电脑系统从经验学习中获取新的知识或技能，增强自身的性能。在处理一些诸如识别、预测、决策等问题时，依赖于相应的算法模型对大规模数据进行处理，得到相关模型，并通过模型进行预测和推理。'

In [66]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
            {"role": "user", "content": "请问，什么是RLHF算法？"}
        ],
        functions=functions,
        function_call="auto",  
    )
response["choices"][0]["message"]

<OpenAIObject at 0x1cfad412890> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "ST_answer",
    "arguments": "{\n  \"q\": \"\u4ec0\u4e48\u662fRLHF\u7b97\u6cd5\"\n}"
  }
}

In [69]:
response["choices"][0]["message"]["content"]

In [71]:
response["choices"][0]["message"]["function_call"]["arguments"]

'{\n  "q": "什么是RLHF算法"\n}'

能够发现Chat模型是能够理解“当你无法回答某个问题时，调用该函数，能够获得答案”这段描述的真实作用，并且会在无法回答时选择调用该函数，并且能够提炼问题并作为参数输入到外部函数中。因此，我们只需要仔细设计这一外部函数，使得其能够顺利调用谷歌搜索API并获得搜索连接，然后再对其内容进行爬取即可。换而言之，借助Function calling功能进行搜索问答机器人是可行的。

&emsp;&emsp;接下来我们就来分别介绍关于谷歌搜索Custom Search API的获取以及其背后可编程搜索引擎的获取方法，并尝试在代码环境中调用API来完成谷歌搜索。

### 二、将谷歌搜索API的获取和使用

#### 1.Custom Search API的获取流程

- 启用谷歌搜索API

&emsp;&emsp;和Gmail API的获取类似，这里我们首先需要在谷歌云API库中搜索谷歌搜索API，并获取谷歌搜索API使用权限，然后查阅相关使用说明文档，并最终在代码环境中实现谷歌搜索。这里哦我们首先登录谷歌云Google Cloud：https://console.cloud.google.com/ 。还是在之前创建的Project内点击API和服务进行谷歌搜索API搜索：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/ebc02ad3df0314a75b029e78378bce3.png" alt="ebc02ad3df0314a75b029e78378bce3" style="zoom:33%;" />

> 首次使用谷歌云并创建Project流程，请参考《Ch.11 借助Google API库进行高效AI应用开发》部分相关内容。

然后点击库，进入谷歌云API库的搜索页面：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/7d675a444b379057b22be7bff3ea88c.png" alt="7d675a444b379057b22be7bff3ea88c" style="zoom:33%;" />

并在其中搜索Google search：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/ca43ea388c6386d6031aeed242025d5.png" alt="ca43ea388c6386d6031aeed242025d5" style="zoom:33%;" />

选择Custom Search API：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/282db8b8a223a1fcdd515bd8f8d709d.png" alt="282db8b8a223a1fcdd515bd8f8d709d" style="zoom:33%;" />

> 此外，有个非常具有类似的API——Google Search Console API。和Custom Search API主要用于获取搜索结果不同，Google Search Console API主要用于查询某网页在谷歌搜索中的排名和浏览情况，即查阅网站运行情况，并据此进一步制定SEO优化措施。

点击启用：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/2793afd92edaa5431490fc41c991abe.png" alt="2793afd92edaa5431490fc41c991abe" style="zoom:33%;" />

稍等片刻即会自动跳转进入如下页面：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/fbbd4ccf096cb99310530cef83a5be0.png" alt="fbbd4ccf096cb99310530cef83a5be0" style="zoom:33%;" />

这里需要注意的是，谷歌搜索API并不是完全免费的API，而是有一定每日免费额度的半收费性质的API，我们可以在配额页面查看具体额度说明：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/d4035bdd0b16a54d83693ad56419004.png" alt="d4035bdd0b16a54d83693ad56419004" style="zoom:33%;" />

这里我们需要重点关注涉及查询类API调用的关键额度，也就是每天100次搜索限制，尽管者并不是一个很高的搜索额度，不过对于学习者以及初期开发测试阶段应用来说，这个免费配额是够用的。当然，如果超出这个限额则需要按照5美元/千次进行计费，且普通开发者账户每天不能超过1万次查询，具体费用情况可以查询相关帮助文档：https://developers.google.com/custom-search/v1/overview 。而如果涉及到支付费用，首先可以按照Ch 11中所说的方法领取免费的试用金（300美元），如果超出300美元赠送金额，则可以进一步通过绑定虚拟信用卡（和OpenAI账户支付类似，详情参考Ch 1中相关内容介绍）进行费用支付。具体绑定银行卡和支付过程，可以在费用一栏进行操作。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/dc6f175cd93a6d5cf9fb1c375bb558f.png" alt="dc6f175cd93a6d5cf9fb1c375bb558f" style="zoom:33%;" />

- 谷歌搜索API的基本调用规则

&emsp;&emsp;在此前的课程中，我们曾详细介绍过Gmail API的调用方式，Gamil是通过OAuth 2.0作为授权凭据，需要开发者和邮箱所有者双方授权方可使用，并且在实际调用Gmail API时也是对授权者的Gmail邮箱进行操作。而在调用谷歌搜索API时，也同样需要授权凭据以及调用的应用对象主体，只不过和Gmail API调用过程不同的是，谷歌搜索的API调用凭据仅需API Key即可，无需使用更高级的OAuth 2.0作为授权凭据，并且调用的应用对象主体也不是谷歌搜索，而是需要自己创建一个基于谷歌搜索的可编程搜索引擎（就相当于申请了一个个人专属的Gmail邮箱作为调用应用的主体），然后再对其进行调用。接下来我们分别介绍谷歌云API的API Key获取方法以及可编程搜索引擎的基本概念与创建流程。

- 谷歌搜索API Key申请流程

&emsp;&emsp;Ch 11课程内容介绍根据此前的介绍，相比OAuth 2.0凭据，API Key凭据其实是一种更加简便的凭据，其本质就是一串字符串密钥，用于调用API时进行身份验证（就类似于OpenAI API Key），使用时正确输入当前项目的密钥即可，并不需要进行更进一步的类似OAuth 2.0凭据的双方授权的环节即可直接使用。

&emsp;&emsp;这里我们可以在项目的凭据页面进行API Key的创建：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/6fbbfbf24d38ffc7486a12b6af5483b.png" alt="6fbbfbf24d38ffc7486a12b6af5483b" style="zoom:33%;" />

然后点击创建凭据——>API密钥：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/6b113f6333911abe2a48fbdb164f2cc.png" alt="6b113f6333911abe2a48fbdb164f2cc" style="zoom:33%;" />

稍等片刻即可完成API Key的密钥创建。和OAuth 2.0凭据定位类似，API Key也是绑定在当前项目中的，即在默认情况下，当我们已经创建了API Key，那么这个API Key是可以访问当前项目中全部可以用API Key作为凭据的API的。因此如果这里有需要，也可以点击下方的修改API密钥以修改其作用的应用范围。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/3940d7d550e22ae8dc56840c095ec6d.png" alt="3940d7d550e22ae8dc56840c095ec6d" style="zoom:33%;" />

当然，为了方便后续调用，以及保密性原则，我们也可以按照类似OpenAI API Key的设置方法，将其设置为环境变量，然后再通过环境变量的变量名直接导入API Key。具体将API Key保存为环境变量的过程可以参考Ch 1中相关说明。这里我们将谷歌搜索API Key的变量名设置为GOOGLE_SEARCH_API_KEY，之后即可通过其调用API Key。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/63540e4a8d49763e80735195ba0b7bb.png" alt="63540e4a8d49763e80735195ba0b7bb" style="zoom:33%;" />

#### 2.可编程搜索引擎获取流程

- 创建可编程搜索引擎

&emsp;&emsp;既然是调用某个应用的API，那么肯定需要“先拥有某个应用”，才能构成接下来API调用的主体，例如Gmail API调用的是Gmail应用，而谷歌搜索API则是调用可编程搜索引擎API。所谓的可编程搜索引擎API，可以将其理解为基于谷歌搜索技术支持的个人定制化搜索引擎，其中“可编程”一词可以理解为是功能上可以一定程度进行定制开发，然后非常便捷的嵌入到其他网页开发项目中，因此可编程的搜索引擎也被称为Google Custom Search Engine（CSE）。尽管听起来非常复杂，但实际上，其申请流程以及使用方法都非常简单，最简单的使用甚至无需进行“代码编程”。这里我们首先使用谷歌账号登录可编程搜索引擎主页进行CSE申请和创建：https://programmablesearchengine.google.com/ ：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/6e2282c31d37a4794536ea35dcf6abb.png" alt="6e2282c31d37a4794536ea35dcf6abb" style="zoom:33%;" />

然后进行这个搜索引擎的个性化配置。其实可选的选项并不多，重点需要关注的是我们可以选择搜索范围。例如，如果我们创建一个专门用于sklearn库使用方法查询的搜索引擎，那么就可以选择在特定网页中进行搜索，并输入sklearn官网网址：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/6358b799c574fdc07f5ad3ef673daa8.png" alt="6358b799c574fdc07f5ad3ef673daa8" style="zoom:33%;" />

然后创建完成后会返回一段JavaScript脚本，后续即可将其嵌入到其他开发项目中：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/9a242bd68a5d0c6c2e9734a35f86beb.png" alt="9a242bd68a5d0c6c2e9734a35f86beb" style="zoom:33%;" />

当然，当前项目并不是考虑将获得的搜索引擎嵌入到某个网页中去，而是希望通过API调用这个搜索引擎API，从而进一步让Chat模型接入搜索引擎。而在使用API调用某个CSE时，需要使用CSE的编号作为身份标识，上段JavaScript脚本中“cx=”之后的字符串就是当前搜索引擎的编号，同样需要将其设置为环境变量方便之后调用。这里我们以CSE_ID作为该变量的变量名：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/612a89e3d41389fa434744cec55de2e.png" alt="612a89e3d41389fa434744cec55de2e" style="zoom:33%;" />

- 更多CSE设置

&emsp;&emsp;当然，在我们创建完一个定制化搜索引擎之后，这个搜索引擎就拥有了一个独立的公开网址，点击即可进入一个极简版的定制化谷歌搜索页面：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1692955413979.png" style="zoom:33%;" />

搜索过程如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1692955480509.png" alt="1692955480509" style="zoom:35%;" />

简单测试搜索引擎效果之后，接下来我们在CSE主页查看当前搜索引擎的基本设置情况。这里可以通过点击蓝色的engine名称，进入当前自定义搜索引擎的设置页面：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/8ab31b60fe6ac8aa1a5df7adcd2a3ad.png" alt="8ab31b60fe6ac8aa1a5df7adcd2a3ad" style="zoom:33%;" />

在该页面中，我们可以查看搜索引擎编号，以及包括外观、开发者团队、广告设置、搜索功能等各项设置。需要注意的是，我们可以在搜索引擎实际使用过程中随时调整搜索引擎设置，而不用重新创建新的搜索引擎。

&emsp;&emsp;至此，我们完整的获取谷歌搜索相关API凭据（以及CSE ID）。总的来看谷歌搜索API的获取和Gmail API获取流程上并没有太大的区别，都是以谷歌云作为中间平台进行授权，然后使用API去调用某应用，所不同的是，Gmail API调用要求OAuth 2.0授权，且调用的是Gmail邮箱，而谷歌搜索API调用只需要API Key即可，同时调用的是自定义的CSE。接下来，我们即可在此基础上尝试调用API来进行谷歌搜索。

#### 3.谷歌搜索API使用与参数介绍

- 谷歌搜索API使用测试

&emsp;&emsp;接下来，我们尝试调用谷歌搜索API，实现在本地代码环境中完成搜索任务。首先导入此前获取的google_search_key和cse_id：

In [3]:
google_search_key = os.getenv("GOOGLE_SEARCH_API_KEY")
cse_id = os.getenv("CSE_ID")

然后设置搜索关键词：

In [42]:
search_term = "OpenAI"

接下来进行搜索。这里需要注意，谷歌搜索API同样也是RESTful风格API，RESTful风格API调用方法详见Ch 10中OpenWeather API相关内容的讲解。总的来说，调用谷歌搜索API同样是分四步进行，分别是构建请求、设置参数、发送请求和解析响应。并且和OpenWeather API一样，谷歌搜索API返回结果也是Json格式对象。这里我们尝试调用API完成谷歌搜索：

In [43]:
import requests

# Step 1.构建请求
url = "https://www.googleapis.com/customsearch/v1"

# Step 2.设置查询参数
params = {
    'q': search_term,           # 搜索关键词
    'key': google_search_key,   # 谷歌搜索API Key
    'cx': cse_id                # CSE ID
}

# Step 3.发送GET请求
response = requests.get(url, params=params)

# Step 4.解析响应
data = response.json()

能够看出，整体调用代码和OpenWeather API调用代码类似，只在url和具体参数方面有所求别。这里的参数部分，q表示搜索关键词、key表示谷歌搜索API Key，而cse_id则表示可编程搜索引擎的ID。除此之外，还有部分后续可能会用到的关键参数解释如下：

|Name|Description|      
|:--:|:--:| 
|c2coff|是否开启中文搜索模式，默认0表示开启，1表示停止| 
|cr|在指定的国家地区进行搜索，即搜索返回结果只源于该地区|
|dateRestrict|只搜索某特定时间段内的内容|
|lr|搜索特定语言内容的内容|
|num|搜索返回的结果个数，默认为10个，有效数值为1-10|
|siteSearch|搜索结果必须包含某网址|
|siteSearchFilter|搜索结果必须排除某网址|

这里需要注意的是，尽管看起来中文搜索或者设置语言为中文会更加有助于阅读，但实际上，我们最终是将搜索结果输入Chat模型进行理解并输出，这里我们只需要让Chat模型最终输出结果是中文就行，搜索的结果可以不是中文。并且经过测试我们发现，若这里设置只进行中文搜索，反而会因为限制了搜索范围而导致搜索结果质量下降。

&emsp;&emsp;除了这些参数外，更多完整参数解释可以参考官网给出的说明文档：https://developers.google.com/custom-search/v1/reference/rest/v1/cse/list ：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/ae636d458ad3d7d6b614e238d1a10fc.png" alt="ae636d458ad3d7d6b614e238d1a10fc" style="zoom:33%;" />

> 不难发现，在调用API时有些参数调整的内容和网页端设置页面调整内容是一样的，例如siteSearch要求搜索结果必须包含某个网址，其实就是要求在某网页内进行搜索。这里需要注意，API调用时参数设置和网页端参数页面的参数设置是相互独立的，即网页端的设置只对利用Web进行搜索时起作用，而调用API时的参数设置，则对代码环境中调用返回结果起作用。

&emsp;&emsp;接下来，尝试解读谷歌搜索API返回的结果。在默认情况下，谷歌搜索会返回10个结果（num=10），且保存在Json对象的items关键词中：

In [44]:
data

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&relatedSite={relatedSite?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - OpenAI',
    'totalResults': '104000000',
    'searchTerms': 'OpenAI',
    'count': 10,
    'startIndex': 1,
    'inputEncoding': 'utf8',
    'outputEncoding': 'utf8',
  

In [45]:
data['items']

[{'kind': 'customsearch#result',
  'title': 'OpenAI',
  'htmlTitle': '<b>OpenAI</b>',
  'link': 'https://openai.com/',
  'displayLink': 'openai.com',
  'snippet': 'Creating safe AGI that benefits all of humanity.',
  'htmlSnippet': 'Creating safe AGI that benefits all of humanity.',
  'cacheId': 'ZJjhy9NmZvMJ',
  'formattedUrl': 'https://openai.com/',
  'htmlFormattedUrl': 'https://<b>openai</b>.com/',
  'pagemap': {'metatags': [{'og:image': 'https://images.openai.com/blob/fb4a2ba6-9109-4c7b-af4d-cae530c3fa78/recruitment-video-poster.jpg?trim=0%2C0%2C0%2C0',
     'og:image:alt': 'People chatting around a couch in a plant-filled sunny room',
     'twitter:card': 'summary_large_image',
     'twitter:site': '@OpenAI',
     'viewport': 'width=device-width, initial-scale=1',
     'og:title': 'OpenAI',
     'og:description': 'Creating safe AGI that benefits all of humanity',
     'twitter:image': 'https://images.openai.com/blob/fb4a2ba6-9109-4c7b-af4d-cae530c3fa78/recruitment-video-poster.jp

这里的items对象是由一系列字典所组成的list，其中每个字典都代表一个搜索结果：

In [46]:
len(data['items'])

10

我们具体查看其中返回的第一个结果：

In [48]:
data['items'][0]

{'kind': 'customsearch#result',
 'title': 'OpenAI',
 'htmlTitle': '<b>OpenAI</b>',
 'link': 'https://openai.com/',
 'displayLink': 'openai.com',
 'snippet': 'Creating safe AGI that benefits all of humanity.',
 'htmlSnippet': 'Creating safe AGI that benefits all of humanity.',
 'cacheId': 'ZJjhy9NmZvMJ',
 'formattedUrl': 'https://openai.com/',
 'htmlFormattedUrl': 'https://<b>openai</b>.com/',
 'pagemap': {'metatags': [{'og:image': 'https://images.openai.com/blob/fb4a2ba6-9109-4c7b-af4d-cae530c3fa78/recruitment-video-poster.jpg?trim=0%2C0%2C0%2C0',
    'og:image:alt': 'People chatting around a couch in a plant-filled sunny room',
    'twitter:card': 'summary_large_image',
    'twitter:site': '@OpenAI',
    'viewport': 'width=device-width, initial-scale=1',
    'og:title': 'OpenAI',
    'og:description': 'Creating safe AGI that benefits all of humanity',
    'twitter:image': 'https://images.openai.com/blob/fb4a2ba6-9109-4c7b-af4d-cae530c3fa78/recruitment-video-poster.jpg?trim=0%2C0%2C0%2

能够发现，每个搜索结果则包括网页标题（title）、地址（link）和网页内容摘要（snippet）等信息。从返回结果来看，搜索OpenAI关键词第一个返回的结果就是OpenAI官网，而第二个返回的结果则是ChatGPT网址：

In [49]:
data['items'][1]

{'kind': 'customsearch#result',
 'title': 'ChatGPT',
 'htmlTitle': 'ChatGPT',
 'link': 'https://chat.openai.com/',
 'displayLink': 'chat.openai.com',
 'snippet': 'ChatGPT is an AI-powered language model developed by OpenAI, capable of generating human-like text based on context and past conversations.',
 'htmlSnippet': 'ChatGPT is an AI-powered language model developed by <b>OpenAI</b>, capable of generating human-like text based on context and past conversations.',
 'cacheId': 'JFIzOqn6hDsJ',
 'formattedUrl': 'https://chat.openai.com/',
 'htmlFormattedUrl': 'https://chat.<b>openai</b>.com/',
 'pagemap': {'metatags': [{'og:image': 'https://chat.openai.com/images/chatgpt-share-og.png',
    'apple-itunes-app': 'app-id=6448311069',
    'next-head-count': '13',
    'viewport': 'width=device-width, initial-scale=1',
    'og:title': 'ChatGPT',
    'title': 'ChatGPT: Get instant answers, find inspiration, learn something new',
    'og:url': 'https://chat.openai.com',
    'og:description': 'A 

总的来看，搜索返回结果还是非常可靠的。更多关于更多谷歌搜索API的使用细则以及响应结果的解释，可以参考谷歌搜索API使用文档：https://developers.google.com/custom-search/v1/using_rest

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/b9e52466e3d6cd0809acb079213dc8c.png" alt="b9e52466e3d6cd0809acb079213dc8c" style="zoom:33%;" />

至此，我们就完成了谷歌搜索API的调用过程。

&emsp;&emsp;不过根据此前的分析不难发现，谷歌搜索API返回的结果实际上是网址而非具体的信息，若要将具体的信息输入到Chat模型中，还需要将网址内的核心信息爬取并输入到模型中去，因此，围绕搜索返回的结果，我们重点提取网页标题（title）、地址（link）和网页内容摘要（snippet）三项信息即可。并且根据此前的介绍，由于我们后续需要重点围绕某特定网站的特定信息进行爬取，因此还需要单独设置site_url参数，用于限制搜索的网页范围。为此我们创建google_search函数如下：

In [2]:
def google_search(query, num_results=10, site_url=None):
    
    api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
    cse_id = os.getenv("CSE_ID")
    
    url = "https://www.googleapis.com/customsearch/v1"

    # API 请求参数
    if site_url == None:
        params = {
        'q': query,          
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results   
        }
    else:
        params = {
        'q': query,         
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results,  
        'siteSearch': site_url
        }

    # 发送请求
    response = requests.get(url, params=params)
    response.raise_for_status()

    # 解析响应
    search_results = response.json().get('items', [])

    # 提取所需信息
    results = [{
        'title': item['title'],
        'link': item['link'],
        'snippet': item['snippet']
    } for item in search_results]

    return results

In [17]:
search_term = "OpenAI"

In [18]:
results = google_search(query=search_term, num_results=5)

In [19]:
results

[{'title': 'OpenAI',
  'link': 'https://openai.com/',
  'snippet': 'Creating safe AGI that benefits all of humanity.'},
 {'title': 'ChatGPT',
  'link': 'https://chat.openai.com/',
  'snippet': 'ChatGPT is an AI-powered language model developed by OpenAI, capable of generating human-like text based on context and past conversations.'},
 {'title': 'Introducing ChatGPT',
  'link': 'https://openai.com/blog/chatgpt',
  'snippet': 'Nov 30, 2022 ... ... and learn about its strengths and weaknesses. During the research preview, usage of ChatGPT is free. Try it now at chat.openai.com.'},
 {'title': 'OpenAI Platform',
  'link': 'https://platform.openai.com/playground',
  'snippet': "Explore resources, tutorials, API docs, and dynamic examples to get the most out of OpenAI's developer platform."},
 {'title': "GPT-4 is OpenAI's most advanced system, producing safer and ...",
  'link': 'https://openai.com/gpt-4',
  'snippet': "Mar 13, 2023 ... GPT-4 is OpenAI's most advanced system, producing safer

接下来我们测试在知乎中搜索RLHF相关信息，得到结果如下：

In [20]:
results = google_search(query='什么是RLHF', num_results=5, site_url='https://www.zhihu.com/')

In [21]:
results

[{'title': '如何看待Geoffrey Hinton对RLHF的看法？ - 知乎',
  'link': 'https://www.zhihu.com/question/589955237',
  'snippet': '或者具体点说，如果目标是让Learning/Optimization/Exploration 本身做的更漂亮更有进步空间，那么RLHF 是有偏的、短视的、可能过拟合于人类喜好的，也就会导致各位\xa0...'},
 {'title': 'RLHF - 知乎',
  'link': 'https://www.zhihu.com/topic/26859901',
  'snippet': 'DPO是最新的最高效的RLHF训练方法。RLHF一直是生成式AI训练的老大难问题，也被认为是OpenAI的压箱底独家秘笈。DPO技术改变了这一切…'},
 {'title': 'ColossalChat：完整RLHF平替ChatGPT的开源方案',
  'link': 'https://www.zhihu.com/tardis/zm/art/618048558?source_id=1005',
  'snippet': '此外，Alpaca的训练数据集仅限于英语，这在一定程度上限制了模型的性能。 然而，ChatGPT和GPT-4令人印象深刻的效果是由于在训练过程中引入了RLHF，这增加了生成的内容\xa0...'},
 {'title': '如何看待Geoffrey Hinton对RLHF的看法？ - Keyu Tian 的回答- 知乎',
  'link': 'https://www.zhihu.com/question/589955237/answer/2943963130',
  'snippet': '有点感触，提供一个辩证的视角~ 评价RLHF 其实可以从目标来出发。我理解Hinton 可能主要是从下面第一个…'},
 {'title': 'ChatGPT 是资本吹起的泡沫吗？相对原有技术真的有那么大的颠覆 ...',
  'link': 'https://www.zhihu.com/question/582688695',
  'snippet': '虽然说没有哪个研究领域，甚至没有哪个领域敢说自己真的是天道酬勤绝对公平， ... ChatGP

能够发现，site_url参数能够正常发挥作用，并返回适当的结果。至此，我们就基本测试了谷歌搜索API的基本测试和函数编写。

### 三、搜索结果信息爬取策略

#### 1.前期准备

- 爬虫方案选取

&emsp;&emsp;接下来，我们进一步考虑围绕这些连接进行内容爬取。由于我们只需要在特定网站内进行搜索和信息爬取，因此本部分爬虫难度相对较低。这里我们考虑采用lxml库中的etree模块来解析HTML，并使用XPath来提取所需的数据。这里之所以没有采用其他更加通用的爬虫框架（如beautifulsoup等工具），主要原因有以下几点：
- 特定网址内的HTML结构相对稳定，因此采用XPath来进行特定文本内容的匹配和提取门槛较低，且运行效果较为稳定，同时也便于维护——若网站的HTML结构发生变化，重新进行定位和匹配即可；
- 通过HTML结构来进行内容提取往往可以非常精准的提取到目标内容，例如本节案例中是想要创建技术方面的搜索问答机器人，那么在进行相关网页爬取时，往往只需要爬取目标文章的标题、正文和代码即可，此时我们只需要在目标网页HTML中查找对应内容的结构即可爬取。而如果采用beautifulsoup等工具则会附带爬取非常多网页中的非必要信息，例如跳转连接名称、企业名称、无关申明等，且过滤这些内容较为繁琐。并且需要注意的是，由于大模型本身存在最大上下文限制，因此在爬取内容时精准的提取“高纯度”内容是非常必要的。（例如以下就是某段beautifulsoup爬取的结果）

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/c2f7b664b44e62d5fc82ebccaea4877.png" alt="c2f7b664b44e62d5fc82ebccaea4877" style="zoom:33%;" />

> HTML（Hyper Text Markup Language），超文本标记语言，我们可以将其通俗的理解为一种描述网页结构的语言，如果将网页比喻成一本书，那么HTML就是用于标记每一部分有哪些内容的目录。

因此，尽管beautifulsoup能够更好的适用于一般网页的泛信息爬取，但在当前项目中更适合使用etree模块来解析HTML，并使用XPath来提取所需内容。

- 爬取内容的本地保存方法

&emsp;&emsp;而在正式进行内容爬取之前，我们还需要提前准备爬取内容本地保存的基本格式。这里我们统一采用JSON格式进行保存，其中一个JSON文件保存一个爬取到的内容（比如一篇文章），每个JSON对象都包含所爬取到的内容的标题（知乎某页的标题，也就是所搜结果中的title）、链接（link）、正文和正文总共的token数量，JSON对象的创建方法如下：

In [72]:
import json

file_path = 'result.json'

# 创建包含JSON对象的列表
json_data = [
    {
        "link": "https://example.com/article/1",
        "title": "Sample Article 1",
        "content": "This is the content of the first article...",
        "tokens": "Number of tokens..."
    }
]

# 指定保存文件的路径
file_path = "result.json"

# 将JSON数据写入本地文件
with open(file_path, "w", encoding="utf-8") as json_file:
    json.dump(json_data, json_file, ensure_ascii=False, indent=4)

print(f"JSON数据已保存到文件：{file_path}")

JSON数据已保存到文件：result.json


在实际爬取过程中，我们设置每个JSON文件名称就是title，同时在每次执行搜索爬取时都会独立创建一个文件夹，用于保存某次搜索时所爬取到的全部文章内容。

#### 2.爬虫编写

##### 2.1 知乎网页结构探索

&emsp;&emsp;接下来我们尝试编写特定知乎网页的爬虫。由于我们采用的XPath匹配的方式进行信息爬取，因此需要先简单确定知乎目标网页的HTML结构，这里需要注意的是，同样的网站内，一级、二级域名对应的网站HTML结构可能会发生很大的变化，且不同功能类型的子网站之间的HTML也会有很大的区别，因此我们需要先仔细观察此前搜索结果中得到的网页列表，来判断知乎网站有哪些不同类型的子网站（以及二级域名网站）：

In [21]:
results

[{'title': '如何看待Geoffrey Hinton对RLHF的看法？ - 知乎',
  'link': 'https://www.zhihu.com/question/589955237',
  'snippet': '或者具体点说，如果目标是让Learning/Optimization/Exploration 本身做的更漂亮更有进步空间，那么RLHF 是有偏的、短视的、可能过拟合于人类喜好的，也就会导致各位\xa0...'},
 {'title': 'RLHF - 知乎',
  'link': 'https://www.zhihu.com/topic/26859901',
  'snippet': 'DPO是最新的最高效的RLHF训练方法。RLHF一直是生成式AI训练的老大难问题，也被认为是OpenAI的压箱底独家秘笈。DPO技术改变了这一切…'},
 {'title': 'ColossalChat：完整RLHF平替ChatGPT的开源方案',
  'link': 'https://www.zhihu.com/tardis/zm/art/618048558?source_id=1005',
  'snippet': '此外，Alpaca的训练数据集仅限于英语，这在一定程度上限制了模型的性能。 然而，ChatGPT和GPT-4令人印象深刻的效果是由于在训练过程中引入了RLHF，这增加了生成的内容\xa0...'},
 {'title': '如何看待Geoffrey Hinton对RLHF的看法？ - Keyu Tian 的回答- 知乎',
  'link': 'https://www.zhihu.com/question/589955237/answer/2943963130',
  'snippet': '有点感触，提供一个辩证的视角~ 评价RLHF 其实可以从目标来出发。我理解Hinton 可能主要是从下面第一个…'},
 {'title': 'ChatGPT 是资本吹起的泡沫吗？相对原有技术真的有那么大的颠覆 ...',
  'link': 'https://www.zhihu.com/question/582688695',
  'snippet': '虽然说没有哪个研究领域，甚至没有哪个领域敢说自己真的是天道酬勤绝对公平， ... ChatGP

我们发现在返回的5个结果中，都是来自一级域名https://www.zhihu.com/ 的部分子页面，并且种类各异，主要有以下几类：

- question页面

&emsp;&emsp;某些问题的主页，例如https://www.zhihu.com/question/589955237 ，页面内容呈现形式如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/a7279f3edf4bbfd661d6a4f6c168270.png" alt="a7279f3edf4bbfd661d6a4f6c168270" style="zoom:33%;" />

对于热门话题，以及通过搜索引擎自然得到的排名较高的问答贴往往都有较高的参考价值，因此是不错的爬取对象。

- question/answer页面

&emsp;&emsp;某个问题的某特定回答置顶页面，例如https://www.zhihu.com/question/589955237/answer/2943963130 ，其内容结构和question页面基本一致，可以作为内容爬取对象。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/962f960f3a6910a67a71457960c5392.png" alt="962f960f3a6910a67a71457960c5392" style="zoom:33%;" />

- topic页面

&emsp;&emsp;话题页面，例如https://www.zhihu.com/topic/26859901 ，呈现形式如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/07544f42d5154a2f877deb4ad14675c.png" alt="07544f42d5154a2f877deb4ad14675c" style="zoom:33%;" />

这种页面的内容的内容较为零散，内容知识纯度较低，并不是爬取知识内容并输入给模型的最佳选择。

&emsp;&emsp;此之外，还有一些子网页也应该排除爬取范围之外，主要有www.zhihu.com/collection 类网站（收藏列表），www.zhihu.com/people 类网站（个人主页），以及www.zhihu.com/column 类网站（个人文章主页）。

&emsp;&emsp;既然这些网站不在爬取范围内，那不妨在搜索引擎的设置中设置过滤这些子网站，以便于更高效的获取目标结果。这里我们可以在可编程搜索引擎主页https://programmablesearchengine.google.com/ 对搜索引擎进行设置，在要排除的网页一栏输入这四类子网页的网址即可：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/3942ecf23f2b9fc036448ee9f8ec003.png" alt="3942ecf23f2b9fc036448ee9f8ec003" style="zoom:33%;" />

此外需要注意的是，还有一类高质量的文章内容在site_url='https://www.zhihu.com/' 模式下无法搜索得到，那就是知乎专栏内容。知乎专栏在单独的一个二级域名https://zhuanlan.zhihu.com/ 内，因此如果希望在搜索的时候同时覆盖问答贴和专栏文章，则需要按照如下参数进行设置site_url='https://zhihu.com/' ，便可在全部一级、二级域名下进行搜索：

In [80]:
results = google_search(query='什么是RLHF', num_results=10, site_url='https://zhihu.com/')

In [81]:
results

[{'title': 'ChatGPT 背后的“功臣”——RLHF 技术详解- 知乎',
  'link': 'https://zhuanlan.zhihu.com/p/599016986',
  'snippet': 'OpenAI 推出的ChatGPT 对话模型掀起了新的AI 热潮，它面对多种多样的问题对答如流，似乎已经打破了机器和人的边界。这一工作的背后是大型语言模型(Large Language\xa0...'},
 {'title': '从零实现ChatGPT——RLHF技术笔记- 知乎',
  'link': 'https://zhuanlan.zhihu.com/p/591474085',
  'snippet': 'Dec 18, 2022 ... 3，用强化学习训练上面那个finetune后的GPT3模型。用强化学习做LM训练的一种思路是用Policy Gradient做，这一块OpenAI用的是他们在17年提出的PPO算法\xa0...'},
 {'title': '如何看待Geoffrey Hinton对RLHF的看法？ - 知乎',
  'link': 'https://www.zhihu.com/question/589955237',
  'snippet': '或者具体点说，如果目标是让Learning/Optimization/Exploration 本身做的更漂亮更有进步空间，那么RLHF 是有偏的、短视的、可能过拟合于人类喜好的，也就会导致各位\xa0...'},
 {'title': '抱抱脸：ChatGPT背后的算法——RLHF | 附12篇RLHF必刷论文- 知乎',
  'link': 'https://zhuanlan.zhihu.com/p/592671478',
  'snippet': 'Dec 18, 2022 ... 但其实这种生成模型很难训练。以语言模型为例，大多是采用“自回归生成”的方式，通过循环解码的方式来逐字或逐词生成内容\xa0...'},
 {'title': 'ChatGPT背后的技术之理解人类反馈强化学习（RLHF） - 知乎',
  'link': 'https://zhuanlan.zhihu.com/p/615708794',
  'snippet': '可以形象解释系PPO近端策略优化算

以此便可获得更加高质量的搜索结果。

##### 2.2 特定网页爬虫编写

&emsp;&emsp;特别说明，在编写和使用爬虫时，请确保始终遵循目标网站的使用条款和隐私政策，课程中介绍的爬虫只用于教学目的以及学习者个人使用，严禁用于其他任何商业用途。并且个人用户在学习期间使用爬虫时，也应当尽量减少爬虫使用次数，以减少目标服务器的负担。

- 编写请求头（header）

&emsp;&emsp;接下来考虑编写代码实现此前制定的爬虫方案，即考虑采用lxml库中的etree模块来解析HTML，并使用XPath来提取所需的数据。在该爬虫方案中，首先我们需要编写一个请求头（header），用于模拟浏览器登录行为，header的一般编写格式如下：

In [86]:
headers = {
    'authority': 'www.zhihu.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'cache-control': 'max-age=0',
    'cookie': "Your cookie",                # 需要手动获取cookie
    'upgrade-insecure-requests': '1',
    'user-agent': 'Your user-agent',        # 手动编写或者选择之后给出的user-agent选项选择其一填写
}

其中headers的基本参数解释如下：

1. **authority**: 通常表示请求的目标主机名。在当前项目中需要视情况填写`www.zhihu.com`或者`zhuanlan.zhihu.com`；

2. **accept**: 告诉服务器客户端能够处理的内容类型。这里列出了多种内容类型，包括HTML、XML和各种图片格式，该参数按照给定内容填写即可；

3. **accept-language**: 告诉服务器客户端的首选语言。这里，首选语言被设置为简体中文（zh-CN），其次是其他中文版本（zh），然后是英文（en）。同样该参数的内容按照给定内容填写即可；

4. **cache-control**: 控制缓存的行为。`max-age=0`通常表示客户端不希望得到缓存的响应，而希望从原始服务器获取一个新的响应。该参数的内容按照给定内容填写即可；

5. **cookie**: 包含服务器之前发送给客户端的cookie。这些cookie可能用于身份验证、会话跟踪或其他目的。这里需要重点注意，建议大家自行获取对应个人浏览器产生的cookie，以避免cookie滥用导致被识别为机器人从而导致封IP；

6. **upgrade-insecure-requests**: 这个头部告诉服务器，如果可能的话，客户端希望使用更安全的协议（如HTTPS）进行通信。建议取值为1即可；

7. **user-agent**: 描述发出请求的客户端的类型。这里，它模拟了一个Chrome浏览器的用户代理字符串。网站有时会根据这个头部提供不同的内容或布局，或者检测是否是爬虫。该参数也需要根据自己实际情况进行编写，接下来会提供一些可选的客户端模拟参数，大家可以选择其中一个作为自己的user-agent。

- cookie获取

&emsp;&emsp;为了更加稳定的使用爬虫，推荐大家自行获取当前客户端的cookie，当然也可以采用Selenium工具自动获取cookie。

&emsp;&emsp;所谓`Cookie` ，本质上是服务器发送给浏览器的一小段数据，浏览器在随后的请求中会将这些数据返回给服务器。它们通常用于识别用户、保存用户的会话状态或其他与用户相关的设置。当你需要从需要登录或有某种会话状态的网站抓取数据时，`Cookie` 通常是必要的。

以下是如何确定 `Cookie` 取值的步骤：

1. **手动登录**：首先，你需要在浏览器中手动访问目标网站并登录知乎。

2. **开发者工具**：登录后，打开浏览器的开发者工具。在 Chrome 和 Firefox 中，你可以右键点击页面，然后选择“检查”或“检查元素”。

3. **网络标签**：在开发者工具中，转到“网络”或“Network”标签。

4. **刷新页面**：在开发者工具打开的情况下，刷新页面。这会捕获所有页面加载过程中的网络请求。

5. **查找请求**：在捕获的请求列表中，找到主要的请求（通常是顶部的第一个，或者是与你的目标URL匹配的请求）。

6. **查看请求头**：点击这个请求，然后查找“请求头”或“Request Headers”部分。

7. **复制Cookie**：在“请求头”部分，你应该能看到一个名为 `Cookie` 的字段。你可以直接复制这个字段的值。

8. **使用在爬虫中**：将复制的 `Cookie` 值用于你的爬虫代码中的请求头。

需要注意的是`Cookie` 通常有有效期，过了有效期它可能会过期。如果你的爬虫在某个时间点突然无法工作，你可能需要重新获取新的 `Cookie`。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/379a144a4be269de28d95bf219b3ccd.png" alt="379a144a4be269de28d95bf219b3ccd" style="zoom:33%;" />

- user-agent填写

&emsp;&emsp;填写个人客户端信息，这里提供几个常见的浏览器的User-Agent字符串，大家可以选择其一填入user-agent字段中：

1. **Chrome (Windows 10)**:
```
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36
```

2. **Firefox (Windows 10)**:
```
Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0
```

3. **Safari (macOS)**:
```
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15
```

4. **Edge (Windows 10)**:
```
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59
```

5. **Chrome (Android)**:
```
Mozilla/5.0 (Linux; Android 10; SM-A205U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.120 Mobile Safari/537.36
```

6. **Safari (iPhone)**:
```
Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1 Mobile/15E148 Safari/604.1
```

在获取了cookie和user-agent之后，需要将其填入，获取完整的header对象。

> 需要注意的是，这里一定需要再次定义header才可以运行之后的代码。为避免cookie滥用，当前ipy文件删除了具体的cookie和user-agent定义过程。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/bcd3c7e2790fc763f899578eb64eacd.png" alt="bcd3c7e2790fc763f899578eb64eacd" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/5b3bd22abd0724eaecd0cbe15a23082.png" alt="5b3bd22abd0724eaecd0cbe15a23082" style="zoom: 33%;" />

当然，这里我们也可以将这两个变量作为系统变量，方便使用更加安全的方式对其进行调用。

In [122]:
cookie = os.getenv('search_cookie')
user_agent = os.getenv('search_user_agent')

- 尝试爬取question子页面

&emsp;&emsp;接下来先尝试爬取question子页面，根据此前的描述，我们需要获取这些子页面的title、正文和代码。这里需要注意的是，不同类型页面的爬取流程会略有区别，这里先介绍question子页面的爬取方法，然后再介绍专栏文章页面内容的爬取方法。

&emsp;&emsp;首先确定爬取的link，这里我们考虑爬取https://www.zhihu.com/question/589955237 链接内的内容：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/a7279f3edf4bbfd661d6a4f6c168270.png" alt="a7279f3edf4bbfd661d6a4f6c168270" style="zoom:33%;" />

In [112]:
url = 'https://www.zhihu.com/question/589955237'

然后构建并发送请求，这里使用 `requests` 库发送一个 GET 请求到指定的 `url`，并获取返回的 HTML 文本内容。：

In [113]:
res = requests.get(url, headers=headers).text

In [114]:
res

'<!doctype html>\n<html lang="zh" data-hairline="true" class="itcauecng" data-theme="light"><head><meta charSet="utf-8"/><title data-rh="true">如何看待Geoffrey Hinton对RLHF的看法？ - 知乎</title><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1"/><meta name="renderer" content="webkit"/><meta name="force-rendering" content="webkit"/><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="google-site-verification" content="FTeR0c8arOPKh8c5DYh_9uu98_zJbaWw53J-Sch9MTg"/><meta data-rh="true" name="description" property="og:description" content="RLHF抚养了一个超级早熟的孩子"/><meta data-rh="true" name="keywords" content="人工智能,深度学习（Deep Learning）,强化学习 (Reinforcement Learning),GPT-4,RLHF"/><link data-rh="true" rel="apple-touch-icon" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.81060cab.png"/><link data-rh="true" rel="apple-touch-icon" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.81060cab.png" sizes="152x152"/><link data-rh

然后解析HTML，使用 `lxml` 的 `etree` 模块解析返回的 HTML 内容，以便后续使用 XPath 进行数据提取。：

In [115]:
res_xpath = etree.HTML(res)

In [116]:
res_xpath

<Element html at 0x207e294df80>

接下来使用 XPath 提取文章的标题:

In [117]:
title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]

In [118]:
title

'如何看待Geoffrey Hinton对RLHF的看法？'

这里需要注意，不同页面的HTML格式不同，这里提取标题（以及之后提取的正文内容）的语法是各不相同的，这里所谓的语法指的是xpath函数内的参数部分内容，也就是xpath表达式。具体应该如何获取对应爬取元素对象的Xpath表达式，可以按照如下方式进行：

以下是使用Google Chrome浏览器查看和获取XPath的步骤，但其他现代浏览器（如Firefox、Edge等）的操作方式也类似：

1. **打开网页**:
   - 在Chrome中打开你想要查看的网页。

2. **打开开发者工具**:
   - 右键点击页面上的任意位置，然后选择“检查”（或“Inspect Element”）。
   - 或者，你可以使用快捷键 `Ctrl + Shift + I`（Windows/Linux）或 `Cmd + Option + I`（Mac）。

3. **使用元素选择器**:
   - 在开发者工具的左上角，你会看到一个鼠标指针图标，这是“选择元素”工具。点击它。
   - 然后，将鼠标悬停在页面上的元素上。你会看到元素被高亮显示，并在开发者工具的“Elements”面板中显示其HTML结构。

4. **获取XPath**:
   - 当你找到想要的元素后，在“Elements”面板中右键点击它。
   - 在弹出的菜单中，选择“Copy” > “Copy XPath”。这将元素的XPath复制到剪贴板。

5. **粘贴XPath**:
   - 接下来即可将复制的XPath粘贴到你的代码或其他地方。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/5cc3c10ab86e1ecec09383a05ffe2ea.png" alt="5cc3c10ab86e1ecec09383a05ffe2ea" style="zoom:33%;" />

需要注意，这里编写的xpath表达式不仅适用于当前页面，也适用于后续其他question页面。

&emsp;&emsp;接下来类似的爬取当前网页的第一个回答的正文：

In [128]:
text = ''
text_d = res_xpath.xpath('//div/div/div/div[2]/div/div[2]/div/div/div[2]/span[1]/div/div/span/p/text()')
for t in text_d:
    txt = str(t).replace('\n', ' ')
    text +=txt

print(text)

5月11日更新：RLHF相关内容除了OpenAI，外界可能很少有人知道ChatGPT模型成功的真正原因，实际上，OpenAI也会对ChatGPT拥有的巨大影响力感到不可思议。这种困惑和惊喜就像工程师们解bug时获得的意外成功：We don't know why, but it works.一种普遍的看法是，ChatGPT没有任何革命性技术，正如Meta 首席AI科学家Yann LeCun所说，“只是一些巧妙的技术组合而已”。当然，听到这话的围观群众不免调侃LeCun这种同行评议是“吃不到葡萄说葡萄酸”，不过，从ChatGPT的缔造者们后续的表态来看，恐怕也不会否认他的“酸话”早在2022年1月，OpenAI发布了另一款GPT-3.5微调版本InstructGPT，这是ChatGPT的“兄弟模型”，如果用标准基准来评估它们的原始技术能力，两个模型之间并没有实质性不同。根据OpenAI科学家们的说法，ChatGPT用的微调方法稍有不同，并且添加了一些对话数据，这让ChatGPT更易上手和易用，由此产生了很大的正面影响新增的对话数据固然重要，不过，，OpenAI联合创始人、研究科学家John Schulman认为，简单来说，强化学习是让研究者像训练狗一样训练AI智能体，并为其做出的正确响应提供奖励，而RLHF的基本思路是，RLHF技术背后的其中一个作者正是John Schulman，很多人不知道的是，作为强化学习大牛，John在这一领域作出过许多重大贡献，例如发明了TRPO算法（信赖域策略优化，Trust Region Policy Optimization）、GAE（广义优势估计，Generalized Advantage Estimation）以及TRPO的后代近端策略优化（Descendant Proximal Policy Optimization），也称PPO算法。值得一提的是，其博士导师是强化学习领域的开拓者Pieter Abbeel，并且也在OpenAI创立初期工作过一段时间。在ChatGPT发布前一个月，John Schulman在Robin Ranjit Singh Chauhan主持的TalkRL播客节目中，详细介绍了RLHF想法的产生源头，InstructGPT以WebGPT的主要思想，并阐述了AI对齐以及对AGI实现的看法。从中，我们也可以看到Ch

而除了专栏外，其他问答贴往往并不包含代码，因此至此我们就完成了第一个类型网站（question类网站）的主标题和置顶回答的答案的爬取。

- 将爬取结果保存在本地json文件中

&emsp;&emsp;接下来，我们将爬取结果按照既定要求在本地进行保存。我们在本地创建一个auto_search文件夹，并在文件夹内创建若干个子文件夹，用于保存不同问题的搜索结果。例如当前搜索的问题是“什么是RLHF”，那么我们就以其作为子文件夹名称，并且将该问题搜索爬取得到的内容保存其内。具体保存过程如下：

In [129]:
text

"5月11日更新：RLHF相关内容除了OpenAI，外界可能很少有人知道ChatGPT模型成功的真正原因，实际上，OpenAI也会对ChatGPT拥有的巨大影响力感到不可思议。这种困惑和惊喜就像工程师们解bug时获得的意外成功：We don't know why, but it works.一种普遍的看法是，ChatGPT没有任何革命性技术，正如Meta 首席AI科学家Yann LeCun所说，“只是一些巧妙的技术组合而已”。当然，听到这话的围观群众不免调侃LeCun这种同行评议是“吃不到葡萄说葡萄酸”，不过，从ChatGPT的缔造者们后续的表态来看，恐怕也不会否认他的“酸话”早在2022年1月，OpenAI发布了另一款GPT-3.5微调版本InstructGPT，这是ChatGPT的“兄弟模型”，如果用标准基准来评估它们的原始技术能力，两个模型之间并没有实质性不同。根据OpenAI科学家们的说法，ChatGPT用的微调方法稍有不同，并且添加了一些对话数据，这让ChatGPT更易上手和易用，由此产生了很大的正面影响新增的对话数据固然重要，不过，，OpenAI联合创始人、研究科学家John Schulman认为，简单来说，强化学习是让研究者像训练狗一样训练AI智能体，并为其做出的正确响应提供奖励，而RLHF的基本思路是，RLHF技术背后的其中一个作者正是John Schulman，很多人不知道的是，作为强化学习大牛，John在这一领域作出过许多重大贡献，例如发明了TRPO算法（信赖域策略优化，Trust Region Policy Optimization）、GAE（广义优势估计，Generalized Advantage Estimation）以及TRPO的后代近端策略优化（Descendant Proximal Policy Optimization），也称PPO算法。值得一提的是，其博士导师是强化学习领域的开拓者Pieter Abbeel，并且也在OpenAI创立初期工作过一段时间。在ChatGPT发布前一个月，John Schulman在Robin Ranjit Singh Chauhan主持的TalkRL播客节目中，详细介绍了RLHF想法的产生源头，InstructGPT以WebGPT的主要思想，并阐述了AI对齐以及对AGI实现的看法。从中，我们也可以看到C

In [120]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [130]:
len(encoding.encode(text))

8768

In [131]:
json_data = [
    {
        "link": url,
        "title": title,
        "content": text,
        "tokens": len(encoding.encode(text))
    }
]

In [132]:
with open('./auto_search/什么是RLHF/%s.json' % title, 'w') as f:
    json.dump(json_data, f)

后续则可以按照如下方式进行提取：

In [133]:
with open('./auto_search/什么是RLHF/%s.json' % title, 'r') as f:
    jd = json.load(f)

In [134]:
jd

[{'link': 'https://www.zhihu.com/question/589955237',
  'title': '如何看待Geoffrey Hinton对RLHF的看法？',
  'content': "5月11日更新：RLHF相关内容除了OpenAI，外界可能很少有人知道ChatGPT模型成功的真正原因，实际上，OpenAI也会对ChatGPT拥有的巨大影响力感到不可思议。这种困惑和惊喜就像工程师们解bug时获得的意外成功：We don't know why, but it works.一种普遍的看法是，ChatGPT没有任何革命性技术，正如Meta 首席AI科学家Yann LeCun所说，“只是一些巧妙的技术组合而已”。当然，听到这话的围观群众不免调侃LeCun这种同行评议是“吃不到葡萄说葡萄酸”，不过，从ChatGPT的缔造者们后续的表态来看，恐怕也不会否认他的“酸话”早在2022年1月，OpenAI发布了另一款GPT-3.5微调版本InstructGPT，这是ChatGPT的“兄弟模型”，如果用标准基准来评估它们的原始技术能力，两个模型之间并没有实质性不同。根据OpenAI科学家们的说法，ChatGPT用的微调方法稍有不同，并且添加了一些对话数据，这让ChatGPT更易上手和易用，由此产生了很大的正面影响新增的对话数据固然重要，不过，，OpenAI联合创始人、研究科学家John Schulman认为，简单来说，强化学习是让研究者像训练狗一样训练AI智能体，并为其做出的正确响应提供奖励，而RLHF的基本思路是，RLHF技术背后的其中一个作者正是John Schulman，很多人不知道的是，作为强化学习大牛，John在这一领域作出过许多重大贡献，例如发明了TRPO算法（信赖域策略优化，Trust Region Policy Optimization）、GAE（广义优势估计，Generalized Advantage Estimation）以及TRPO的后代近端策略优化（Descendant Proximal Policy Optimization），也称PPO算法。值得一提的是，其博士导师是强化学习领域的开拓者Pieter Abbeel，并且也在OpenAI创立初期工作过一段时间。在ChatGPT发布前一个月，John Schulman在Rob

- 其他类型网站的xpath语法

&emsp;&emsp;而除了要爬取question类网站之外，我们还需要爬取question/answer类网站和专栏类网站的内容，剩余两类网站的xpath语法如下：

In [106]:
# question/answer网站
url = 'https://www.zhihu.com/question/589955237/answer/2943963130'
res = requests.get(url, headers=headers).text
res_xpath = etree.HTML(res)

In [107]:
# question/answer网站标题
res_xpath.xpath('//div/div[1]/div/h1/text()')[0]

'如何看待Geoffrey Hinton对RLHF的看法？'

In [108]:
# question/answer网站置顶回答内容
dic = []
text_d = res_xpath.xpath('//div[1]/div/div[3]/div/div/div/div[2]/span[1]/div/div/span/p/text()')
for t in text_d:
    txt = str(t).replace('\n', ' ')
    dic.append(txt)
    print(txt)

有点感触，提供一个辩证的视角~ 评价 RLHF 其实可以从
来出发。我理解 Hinton 可能主要是从下面
角度考虑的：
或者具体点说，如果目标是让 Learning/Optimization/Exploration 本身做的
，那么 RLHF 是
、可能
于人类喜好的，也就会导致各位老师们提到的“催熟GPT”、容易让人们
于这种过拟合等等后果。
也就有点儿类似于 AlphaGo 最早版本，它依赖从
中学习围棋，那么棋力顶天也就和人类齐平，难以突破。而后续 AlphaZero 不再需要棋谱，从零摸索围棋，那么它超过人类上限、臻入化境也就变得很自然了。
追求机器智能，我们应该是希望它的思考的
或思考
“像”人，而不只是追求它思考的
像”人（RLHF 只能做到后者）
从类似做一个产品的角度，RLHF 是成功的，让我们看到了现代深度学习可以训练出一个基本能通过
的模型。当然可以批评说 RLHF 是有偏的，但这种归纳偏置其实
：例如我们就是想要 ChatGPT 能够拟合人类聊天的语言或偏好，那么通过图灵测试、提供很高的应用价值（写文案、查资料等）无疑就是证据，也突破了我们过往对大语言模型，或者深度学习能到达上限的理解。
最后瞎想一下：RLHF/GPT大模型最终会走向通用智能 AGI 吗？没有人知道，但个人目前稍有担忧：智能真的可以基于统计规律实现吗？至少人思考时的很多逻辑原理看上去不是“RLHF”或“大模型涌现”可以结构化表出的，RLHF/大模型现在只是结果“像”人而已。


In [156]:
# 专栏类网站
url = 'https://zhuanlan.zhihu.com/p/599016986'
res = requests.get(url, headers=headers).text
res_xpath = etree.HTML(res)

In [157]:
# 专栏类网站标题
res_xpath.xpath('//div[1]/div/main/div/article/header/h1/text()')[0]

'ChatGPT 背后的“功臣”——RLHF 技术详解'

In [158]:
# 专栏类网站标题内容
dic = []
text_d = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div/p/text()')
for t in text_d:
    txt = str(t).replace('\n', ' ')
    dic.append(txt)
    print(txt)

OpenAI 推出的 ChatGPT 对话模型掀起了新的 AI 热潮，它面对多种多样的问题对答如流，似乎已经打破了机器和人的边界。这一工作的背后是大型语言模型 (Large Language Model，LLM) 生成领域的新训练范式：RLHF (Reinforcement Learning from Human Feedback) ，即以强化学习方式依据人类反馈优化语言模型。
过去几年里各种 LLM 根据人类输入提示 (prompt) 生成多样化文本的能力令人印象深刻。然而，对生成结果的评估是主观和依赖上下文的，例如，我们希望模型生成一个有创意的故事、一段真实的信息性文本，或者是可执行的代码片段，这些结果难以用现有的基于规则的文本生成指标 (如 BLUE 和 ROUGE) 来衡量。除了评估指标，现有的模型通常以预测下一个单词的方式和简单的损失函数 (如交叉熵) 来建模，没有显式地引入人的偏好和主观意见。
如果我们 
，那不是更好吗？这就是 RLHF 的思想：使用强化学习的方式直接优化带有人类反馈的语言模型。RLHF 使得在一般文本数据语料库上训练的语言模型能和复杂的人类价值观对齐。
看看 ChatGPT 是如何解释 RLHF 的：
ChatGPT 解释的很好，但还没有完全讲透；让我们更具体一点吧！
RLHF 是一项涉及多个模型和不同训练阶段的复杂概念，这里我们按三个步骤分解：
首先，我们使用经典的预训练目标训练一个语言模型。对这一步的模型，OpenAI 在其第一个流行的 RLHF 模型 InstructGPT 中使用了较小版本的 GPT-3; Anthropic 使用了 1000 万 ～ 520 亿参数的 Transformer 模型进行训练；DeepMind 使用了自家的 2800 亿参数模型 Gopher。
这里可以用额外的文本或者条件对这个 LM 进行微调，例如 OpenAI 对 “更可取” (preferable) 的人工生成文本进行了微调，而 Anthropic 按 “有用、诚实和无害” 的标准在上下文线索上蒸馏了原始的 LM。这里或许使用了昂贵的增强数据，但并不是 RLHF 必须的一步。由于 RLHF 还是一个尚待探索的领域，对于” 哪种模型” 适合作为 RLHF 的起点并没有明确的答案。
接下来，我们会基于 LM 来生成训练奖励模型 (RM，也叫偏好

当然，这里需要注意的是，如果是爬取专栏类网站，最好修改'authority': 'zhuanlan.zhihu.com'。

- 自动爬取目标网站函数编写

&emsp;&emsp;接下来我们就尝试编写一个完整的函数，能够爬取目标网站的questions页面、questions/answer页面和专栏页面，具体函数定义过程如下：

In [103]:
cookie = os.getenv('search_cookie')
user_angent = os.getenv('search_user_angent')

In [140]:
url = 'https://www.zhihu.com/question/589955237/answer/2943963130'

In [142]:
if 'answer' in url:
    print('a')

a


In [143]:
headers

{'authority': 'www.zhihu.com',
 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
 'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
 'cache-control': 'max-age=0',
 'cookie': '__snaker__id=bwnbT3V3KsLv9O4P; SESSIONID=k9QurUSnseRdfnxO73qVB6Q1EkEgGYCFDNOhqjHB0Qi; JOID=V1kQBk0LYFnjh5DDVgs3wTCftvhBbjNvk9zAtjdqCG2T1en_NfBpsIeAkMdVAUzz6Hj97hxu_6CQigvv0cGjnMA=; osd=UV0WC0INZF_uiJbHUAY4xzSZu_dHajVinNrEsDplDmmV2Ob5MfZkv4GElspaB0j15Xf76hpj8KaUjAbg18Wlkc8=; _zap=763f6226-49dd-4339-97db-caa75c3e0f68; d_c0=ALCY-hkmPBePTlblae7e2JhfG_ZplrTfURE=|1691953511; YD00517437729195%3AWM_TID=3FO7SjH9L95BAUUQFVaFw%2FoVZIagIEQn; _xsrf=IWtwZurAv7LiGRXc1fAaIH1uARD8Ukoe; captcha_ticket_v2=2|1:0|10:1693023024|17:captcha_ticket_v2|704:eyJ2YWxpZGF0ZSI6IkNOMzFfZGl0UzJMVlltSFZmTGNKamVmaTlUdEhNbkpSMFNnSFZPZTZwYi5VYzRwUjJONi1YcnBXUVFhQjJBNXB1X1Zncms4RHlWYkk0TXcxSWRVT3I5Wm5jOXB4b2k1QUZJdjlfUlhmTkd1WTZfRFJrUElTX0RXdy1rOUZxTVdoSFRf

In [230]:
def get_search_text(q, url):

    cookie = os.getenv('search_cookie')
    user_agent = os.getenv('search_user_angent')    

    code_ = False
    headers = {
        'authority': 'www.zhihu.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': cookie,
        'upgrade-insecure-requests': '1',
        'user-agent':user_agent,
    }

    # 普通问答地址
    if 'zhihu.com/question' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div/div/div/div[2]/div/div[2]/div/div/div[2]/span[1]/div/div/span/p/text()')
    
    # 专栏地址
    elif 'zhuanlan' in url:
        headers['authority'] = 'zhaunlan.zhihu.com'
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div[1]/div/main/div/article/header/h1/text()')[0]
        text_d = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div/p/text()')
        code_ = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div//pre/code/text()')  
            
    # 特定回答的问答网址
    elif 'answer' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div[1]/div/div[3]/div/div/div/div[2]/span[1]/div/div/span/p/text()')

    # 创建问题答案正文
    text = ''
    for t in text_d:
        txt = str(t).replace('\n', ' ')
        text += txt
        
    # 如果有code，则将code追加到正文的追后面
    if code_:
        for c in code_:
            co = str(c).replace('\n', ' ')    
            text += co
    
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")     
    json_data = [
        {
            "link": url,
            "title": title,
            "content": text,
            "tokens": len(encoding.encode(text))
        }
    ]

    with open('./auto_search/%s/%s.json' % (q, title), 'w') as f:
        json.dump(json_data, f)
    
    return title

In [160]:
url = 'https://zhuanlan.zhihu.com/p/599016986'

In [161]:
title = get_search_text(q='什么是RLHF', url=url)
title

'ChatGPT 背后的“功臣”——RLHF 技术详解'

In [162]:
with open('./auto_search/什么是RLHF/%s.json' % 'ChatGPT 背后的“功臣”——RLHF 技术详解', 'r') as f:
    jd = json.load(f)

In [163]:
jd

[{'link': 'https://zhuanlan.zhihu.com/p/599016986',
  'title': 'ChatGPT 背后的“功臣”——RLHF 技术详解',
  'content': 'OpenAI 推出的 ChatGPT 对话模型掀起了新的 AI 热潮，它面对多种多样的问题对答如流，似乎已经打破了机器和人的边界。这一工作的背后是大型语言模型 (Large Language Model，LLM) 生成领域的新训练范式：RLHF (Reinforcement Learning from Human Feedback) ，即以强化学习方式依据人类反馈优化语言模型。过去几年里各种 LLM 根据人类输入提示 (prompt) 生成多样化文本的能力令人印象深刻。然而，对生成结果的评估是主观和依赖上下文的，例如，我们希望模型生成一个有创意的故事、一段真实的信息性文本，或者是可执行的代码片段，这些结果难以用现有的基于规则的文本生成指标 (如 BLUE 和 ROUGE) 来衡量。除了评估指标，现有的模型通常以预测下一个单词的方式和简单的损失函数 (如交叉熵) 来建模，没有显式地引入人的偏好和主观意见。如果我们 ，那不是更好吗？这就是 RLHF 的思想：使用强化学习的方式直接优化带有人类反馈的语言模型。RLHF 使得在一般文本数据语料库上训练的语言模型能和复杂的人类价值观对齐。看看 ChatGPT 是如何解释 RLHF 的：ChatGPT 解释的很好，但还没有完全讲透；让我们更具体一点吧！RLHF 是一项涉及多个模型和不同训练阶段的复杂概念，这里我们按三个步骤分解：首先，我们使用经典的预训练目标训练一个语言模型。对这一步的模型，OpenAI 在其第一个流行的 RLHF 模型 InstructGPT 中使用了较小版本的 GPT-3; Anthropic 使用了 1000 万 ～ 520 亿参数的 Transformer 模型进行训练；DeepMind 使用了自家的 2800 亿参数模型 Gopher。这里可以用额外的文本或者条件对这个 LM 进行微调，例如 OpenAI 对 “更可取” (preferable) 的人工生成文本进行了微调，而 Anthropic 按 “有用、诚实和无害” 的标准在上下文线索上蒸馏了原始的 LM。这里或许使用了昂贵的增强数据，但

至此，我们就完整创建了爬取目标网址的爬虫。接下来我们就将把这个爬虫函数写入外部函数中，构建一个完整的自动搜索问答机器人。

### 四、自动搜索问答机器人构建

#### 1.基本自动问答机器人实现流程

&emsp;&emsp;接下来，我们即可将上述过程整合在一个外部函数中，并借助此前介绍的Function calling功能实现自动搜索问答机器人的构建。

In [165]:
results

[{'title': 'ChatGPT 背后的“功臣”——RLHF 技术详解- 知乎',
  'link': 'https://zhuanlan.zhihu.com/p/599016986',
  'snippet': 'OpenAI 推出的ChatGPT 对话模型掀起了新的AI 热潮，它面对多种多样的问题对答如流，似乎已经打破了机器和人的边界。这一工作的背后是大型语言模型(Large Language\xa0...'},
 {'title': '从零实现ChatGPT——RLHF技术笔记- 知乎',
  'link': 'https://zhuanlan.zhihu.com/p/591474085',
  'snippet': 'Dec 18, 2022 ... 3，用强化学习训练上面那个finetune后的GPT3模型。用强化学习做LM训练的一种思路是用Policy Gradient做，这一块OpenAI用的是他们在17年提出的PPO算法\xa0...'},
 {'title': '如何看待Geoffrey Hinton对RLHF的看法？ - 知乎',
  'link': 'https://www.zhihu.com/question/589955237',
  'snippet': '或者具体点说，如果目标是让Learning/Optimization/Exploration 本身做的更漂亮更有进步空间，那么RLHF 是有偏的、短视的、可能过拟合于人类喜好的，也就会导致各位\xa0...'},
 {'title': '抱抱脸：ChatGPT背后的算法——RLHF | 附12篇RLHF必刷论文- 知乎',
  'link': 'https://zhuanlan.zhihu.com/p/592671478',
  'snippet': 'Dec 18, 2022 ... 但其实这种生成模型很难训练。以语言模型为例，大多是采用“自回归生成”的方式，通过循环解码的方式来逐字或逐词生成内容\xa0...'},
 {'title': 'ChatGPT背后的技术之理解人类反馈强化学习（RLHF） - 知乎',
  'link': 'https://zhuanlan.zhihu.com/p/615708794',
  'snippet': '可以形象解释系PPO近端策略优化算

首先尝试进行外部函数编写：

In [168]:
for item in results:
    print(type(item['link']))
    break

<class 'str'>


In [173]:
jd[0]['tokens']

3337

In [4]:
def get_answer(q):
    """
    当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 默认搜索返回10个答案
    results = google_search(query=q, num_results=10, site_url='https://zhihu.com/')
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    return(content)

然后测试外部函数效果：

In [197]:
q = 'GPT-3.5的微调方法'

In [198]:
qa = get_answer(q)
qa

'一夜之间，大模型界又炸出个big news！斯坦福发布（羊驼，网友口中的“草泥马”）：还有一个更绝的“骚操作”。研究所涉及到的数据集，是斯坦福团队花了不到500美元用OpenAI的API来生成的。所以整个过程下来，就等同于GPT-3.5自己教出了个旗鼓相当的对手AI。（薅羊毛高手……）然后团队还说，用大多数云计算平台去微调训练好的模型，成本也不到100美元：而且团队还把数据集（秒省500刀）、代码统统都给开源了，这下子人人都能去微调个效果炸裂的对话AI：项目在GitHub发布才半天时间，便已经狂揽1800+星，火爆程度可见一斑。Django联合开发者甚至对斯坦福的新研究用“惊天大事”来形容：不仅如此，斯坦福团队还搞了个demo，在线可玩的那种。话不多说，我们现在就来看看这个“草泥马”的效果。在斯坦福官方的演示中，他们先小试牛刀地提了一个问题：草泥马Aplaca给出的答案较为干练：而后又简单的介绍了二者群居生活的不同。同样的问题若是交给ChatGPT（GPT3.5-turbo），则答案就不会像草泥马Aplaca那般简洁：对此，团队给出的解释是：而后团队演示了让草泥马Alpaca：草泥马Alpaca对于这个任务也是信手拈来，直接给出了一个像模像样的邮件模板：难度再次进阶，团队这次提出了让草泥马Alpaca的需求：草泥马Alpaca给出的答案从内容上来看，非常符合大多数论文的摘要形式：试图回答什么问题、用了什么方法、结果如何，以及未来展望。当然，也有迫不及待的网友亲自下场试验，发现草泥马Alpaca写代码也是不在话下。不过即便草泥马Alpaca能够hold住大部分问题，但这并不意味着它没有缺陷。例如团队便演示了一个例子，在回答“坦桑尼亚的首都是哪里”的问题时，草泥马Alpaca给出的答案是“达累斯萨拉姆”。但实际上早在1975年便被“多多马”取代了。除此之外，若是亲自体验过草泥马Alpaca就会发现，它……巨慢：对此，有网友认为可能是使用的人太多的原因。Meta开源的LLaMA大模型，刚发布几周就被大家安排明白了，单卡就能运行。所以理论上，基于LLaMA微调的Alpaca同样可以轻松在本地部署。没有显卡也没关系，苹果笔记本甚至树莓派、手机都可以玩。在苹果笔记本部署LLaMA的方法来自GitHub项目llama.cpp，使用纯C/C++做推理，还专门对ARM芯片做了优化。

In [200]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k-0613",
  messages=[
    {"role": "system", "content": qa},
    {"role": "user", "content": 'GPT-3.5的微调方法'}
  ]
)

In [201]:
response.choices[0].message['content']

'GPT-3.5可以通过微调进行定制，以在特定任务中获得更好的性能。微调是指使用基于任务的数据集对预先训练的模型进行再训练，以使其适应特定的应用场景。下面是GPT-3.5模型的微调方法：\n\n1. 准备数据：您需要根据您的特定任务创建一个数据集。这个数据集应该包含输入和相应的输出，以训练模型在给定输入时生成正确的输出。确保数据集具有足够的多样性和覆盖范围，以使模型能够适应各种情况。\n\n2. 上传文件：将准备好的数据集上传到您选择的云计算平台。确保您的数据集在训练过程中始终可用，并保持安全。\n\n3. 创建微调任务：使用云计算平台的API，创建一个微调任务，并将数据集与任务关联。指定训练模型的参数和超参数，并确定训练过程中的其他设置。确保您的任务配置符合您的需求，并根据需要进行相应的调整。\n\n4. 使用微调模型：一旦模型完成微调，您就可以使用它来进行预测和生成。将输入提供给模型，并获得其生成的输出作为响应。确保对结果进行适当的评估和后处理，以确保其质量和正确性。\n\n请注意，微调的过程需要一定的计算资源和时间，并且在整个过程中需要不断优化和调整。正确使用微调方法可以使GPT-3.5模型在特定任务中表现更好，但仍然需要深入研究和实践。'

能够发现，当前外部函数的返回结果能够非常好的指导模型进行知识库之外的问题的问答。接下来我们将get_answer函数返回结果修改为json对象，然后尝试使用此前定义的run_conversation来执行自动function calling功能：

In [67]:
def get_answer(q):
    """
    当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 默认搜索返回10个答案
    results = google_search(query=q, num_results=10, site_url='https://zhihu.com/')
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    return json.dumps({"Question": q, "Answer": content})

In [232]:
q = 'RLHF总共有几个训练阶段'

In [233]:
get_answer(q)

'{"Question": "RLHF\\u603b\\u5171\\u6709\\u51e0\\u4e2a\\u8bad\\u7ec3\\u9636\\u6bb5", "Answer": "\\u8fd1\\u65e5\\u6765\\uff0cChatGPT\\u53ca\\u7c7b\\u4f3c\\u6a21\\u578b\\u5f15\\u53d1\\u4e86\\u4eba\\u5de5\\u667a\\u80fd\\uff08AI\\uff09\\u9886\\u57df\\u7684\\u4e00\\u573a\\u98ce\\u6f6e\\u3002 \\u8fd9\\u573a\\u98ce\\u6f6e\\u5bf9\\u6570\\u5b57\\u4e16\\u754c\\u4ea7\\u751f\\u4e86\\u9769\\u547d\\u6027\\u5f71\\u54cd\\u3002ChatGPT\\u7c7b\\u6a21\\u578b\\u5177\\u6709\\u60ca\\u4eba\\u7684\\u6cdb\\u7528\\u6027\\uff0c\\u80fd\\u591f\\u6267\\u884c\\u5f52\\u7eb3\\u3001\\u7f16\\u7a0b\\u3001\\u7ffb\\u8bd1\\u7b49\\u4efb\\u52a1\\uff0c\\u5176\\u7ed3\\u679c\\u4e0e\\u4eba\\u7c7b\\u4e13\\u5bb6\\u76f8\\u5f53\\u751a\\u81f3\\u66f4\\u4f18\\u3002\\u4e3a\\u4e86\\u4f7fChatGPT\\u7b49\\u6a21\\u578b\\u7684\\u8bad\\u7ec3\\u548c\\u90e8\\u7f72\\u66f4\\u8f7b\\u677e\\uff0cAI \\u5f00\\u6e90\\u793e\\u533a\\u8fdb\\u884c\\u4e86\\u5404\\u79cd\\u5c1d\\u8bd5\\uff08\\u4f8b\\u5982 ChatLLaMa\\u3001Alpaca\\u3001Vicuna\\u3001Databricks-Doll

直接使用run_conversation测试能否自动调用function calling来执行

In [224]:
run_conversation(messages=[{"role": "user", "content": '请帮我介绍下RLHF算法'}], 
                 functions_list=[get_answer], 
                 model="gpt-3.5-turbo-16k-0613")

'RLHF（Reinforcement Learning from Human Feedback）算法是一种强化学习算法，它的目标是利用人类反馈来加速强化学习的训练过程。\n\n在传统的强化学习中，智能体通过与环境的交互来学习最优策略。然而，这种交互通常需要大量的试错过程，而且在初始阶段可能会出现低效的策略。为了解决这个问题，RLHF算法引入了人类反馈，通过人类提供的知识和经验来指导强化学习的训练过程。\n\nRLHF算法的基本思想是，在智能体与环境的交互过程中，人类观察智能体的行为并提供反馈。这个反馈可以是正式的指导信号，比如给出动作的奖励和惩罚，也可以是非正式的指导信号，比如提供关于行为的建议或者示范。通过将人类反馈和环境奖励相结合，RLHF算法能够更快地学习到高效的策略。\n\n在RLHF算法中，人类反馈被建模为一个辅助的奖励函数。这个奖励函数可以根据人类反馈的类型进行调整，比如可以将正式的指导信号转化为奖励值，或者将非正式的指导信号转化为一个类似奖励值的信号。通过将环境奖励和人类反馈的奖励相加，智能体可以根据这个综合奖励进行策略的更新和优化。\n\nRLHF算法在实际应用中具有广泛的应用。例如，在机器人控制、游戏玩法优化和推荐系统等领域，RLHF算法能够利用人类反馈来改进智能体的性能和效果。\n\n总之，RLHF算法是一种利用人类反馈指导强化学习的算法，通过将人类反馈作为额外的奖励信号，可以加速训练过程并改善智能体的性能。'

In [242]:
run_conversation(messages=[{"role": "user", "content": '请问RLHF算法总共有几个阶段？'}], 
                 functions_list=[get_answer], 
                 model="gpt-3.5-turbo-16k-0613")

'RLHF（Reinforcement Learning from Human Feedback）总共有三个训练阶段。\n\n第一个阶段是基于预训练模型的初始训练（InstructGPT），使用预训练的模型进行强化学习。在这个阶段，模型通过与人类专家对话进行训练，以学习生成更好的回复。\n\n第二个阶段是引入扩展训练（Expert Model Adversarial Training），在初始训练的基础上，引入对抗训练，以提升模型的对抗鲁棒性。通过与专家模型对话，模型可以学习更好地应对不同情况下的回复。\n\n第三个阶段是增强训练（RL Training），在引入扩展训练的基础上，使用强化学习算法对模型进行训练。在这个阶段，模型通过交互式对话与环境进行交互，并使用奖励模型来指导模型进行优化。通过增强训练，模型可以不断改进自己的回答质量和效率。\n\n这三个阶段的训练是紧密相连的，每个阶段都对于RLHF的不同方面和效果起到关键作用。'

至此，即可验证基于function calling的自动搜索问答机器人本身的可行性。

#### 2.全自动搜索问答机器人流程优化

&emsp;&emsp;在能够顺利跑通自动搜索问答机器人的基本代码流程之后，接下来我们考虑围绕该过程进行稳定性和问答性能方面的优化。

- 修改title字符串格式

&emsp;&emsp;首先是关于运行稳定性方面的优化，在此前的代码流程中，相关搜索和爬取到的文档都需要进行本地保存，而在进行本地保存时我们规定每个页面的title就是对应json文档的名称，但有时会遇到文档title存在特殊字符（如？、\*等）而导致无法创建同名文档的问题，因此为了更加稳定的执行自动搜索问答流程，我们需要创建一个title字符过滤函数，用于能够顺利输出符合文档创建格式要求的title字符串：

In [11]:
def windows_compatible_name(s, max_length=255):
    """
    将字符串转化为符合Windows文件/文件夹命名规范的名称。
    
    参数:
    - s (str): 输入的字符串。
    - max_length (int): 输出字符串的最大长度，默认为255。
    
    返回:
    - str: 一个可以安全用作Windows文件/文件夹名称的字符串。
    """

    # Windows文件/文件夹名称中不允许的字符列表
    forbidden_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']

    # 使用下划线替换不允许的字符
    for char in forbidden_chars:
        s = s.replace(char, '_')

    # 删除尾部的空格或点
    s = s.rstrip(' .')

    # 检查是否存在以下不允许被用于文档名称的关键词，如果有的话则替换为下划线
    reserved_names = ["CON", "PRN", "AUX", "NUL", "COM1", "COM2", "COM3", "COM4", "COM5", "COM6", "COM7", "COM8", "COM9", 
                      "LPT1", "LPT2", "LPT3", "LPT4", "LPT5", "LPT6", "LPT7", "LPT8", "LPT9"]
    if s.upper() in reserved_names:
        s += '_'

    # 如果字符串过长，进行截断
    if len(s) > max_length:
        s = s[:max_length]

    return s

In [9]:
# 测试
input_str = "这是一个包含不允许的字符，如 <, >, :, \, /, |, ?, * 等的很长的字符串..."
print(windows_compatible_name(input_str))

这是一个包含不允许的字符，如 _, _, _, _, _, _, _, _ 等的很长的字符串


接下来即可修改get_search_text函数，借助windows_compatible_name修改获取到的title：

In [231]:
def get_search_text(q, url):

    cookis = os.getenv('search_cookie')
    user_agent = os.getenv('search_user_agent')    
    title = None
    
    code_ = False
    headers = {
        'authority': 'www.zhihu.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': cookie,
        'upgrade-insecure-requests': '1',
        'user-agent':user_agent,
    }

    # 普通问答地址
    if 'zhihu.com/question' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div/div/div/div[2]/div/div[2]/div/div/div[2]/span[1]/div/div/span/p/text()')
    
    # 专栏地址
    elif 'zhuanlan' in url:
        headers['authority'] = 'zhaunlan.zhihu.com'
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div[1]/div/main/div/article/header/h1/text()')[0]
        text_d = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div/p/text()')
        code_ = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div//pre/code/text()')  
            
    # 特定回答的问答网址
    elif 'answer' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div[1]/div/div[3]/div/div/div/div[2]/span[1]/div/div/span/p/text()')

    if title == None:
        return None
    
    else:
        title = windows_compatible_name(title)

        # 创建问题答案正文
        text = ''
        for t in text_d:
            txt = str(t).replace('\n', ' ')
            text += txt

        # 如果有code，则将code追加到正文的追后面
        if code_:
            for c in code_:
                co = str(c).replace('\n', ' ')    
                text += co

        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")     
        json_data = [
            {
                "link": url,
                "title": title,
                "content": text,
                "tokens": len(encoding.encode(text))
            }
        ]

        with open('./auto_search/%s/%s.json' % (q, title), 'w') as f:
            json.dump(json_data, f)

        return title

- 创建搜索判别器

&emsp;&emsp;其次在反复多次试验过程中，我们发现，GPT-3.5系列模型在认知get_answer外部函数功能准确性不如GPT-4模型，即在大多数情况下，GPT-4能够在清楚“自己不知道”的情况下自动使用function calling调用get_answer模型，而GPT-3.5系列模型则偶尔会出现幻觉，会胡诌一套问题的答案：

In [14]:
functions_list = [get_answer]

In [15]:
functions = auto_functions(functions_list)
functions

[{'name': 'get_answer',
  'description': '当你无法回答某个问题时，调用该函数，能够获得答案',
  'parameters': {'type': 'object',
   'properties': {'q': {'type': 'string', 'description': '询问的问题'}},
   'required': ['q']}}]

In [16]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
            {"role": "user", "content": "请问GPT-3.5微调总共有几个阶段？"}
        ],
        functions=functions,
        function_call="auto",  
    )
response["choices"][0]["message"]

<OpenAIObject at 0x202e5d3b290> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_answer",
    "arguments": "{\n  \"q\": \"GPT-3.5\u5fae\u8c03\u603b\u5171\u6709\u51e0\u4e2a\u9636\u6bb5\uff1f\"\n}"
  }
}

In [19]:
s = "GPT-3.5\u5fae\u8c03\u603b\u5171\u6709\u51e0\u4e2a\u9636\u6bb5\uff1f"
print(s)

GPT-3.5微调总共有几个阶段？


能够发现，GPT-4模型对外部功能识别能力较强，而GPT-3.5模型则偶尔会出现幻觉：

In [20]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        messages=[
            {"role": "user", "content": "请问GPT-3.5微调总共有几个阶段？"}
        ],
        functions=functions,
        function_call="auto",  
    )
response["choices"][0]["message"]

<OpenAIObject at 0x202e5d3bc40> JSON: {
  "role": "assistant",
  "content": "GPT-3.5\u6a21\u578b\u5fae\u8c03\u901a\u5e38\u5206\u4e3a\u4e24\u4e2a\u9636\u6bb5\u3002\u7b2c\u4e00\u9636\u6bb5\u662f\u9884\u8bad\u7ec3\u9636\u6bb5\uff0c\u6a21\u578b\u5728\u5927\u89c4\u6a21\u7684\u65e0\u76d1\u7763\u6570\u636e\u4e0a\u8fdb\u884c\u9884\u8bad\u7ec3\u3002\u7b2c\u4e8c\u9636\u6bb5\u662f\u5fae\u8c03\u9636\u6bb5\uff0c\u6a21\u578b\u4f7f\u7528\u5c11\u91cf\u6709\u6807\u7b7e\u7684\u6570\u636e\u96c6\u8fdb\u884c\u6709\u76d1\u7763\u5fae\u8c03\uff0c\u4ee5\u9002\u5e94\u7279\u5b9a\u7684\u4efb\u52a1\u3002"
}

In [21]:
response["choices"][0]["message"]["content"]

'GPT-3.5模型微调通常分为两个阶段。第一阶段是预训练阶段，模型在大规模的无监督数据上进行预训练。第二阶段是微调阶段，模型使用少量有标签的数据集进行有监督微调，以适应特定的任务。'

能够发现，GPT-4模型在Function calling功能使用方面确实比GPT-3.5模型“能力更强”，能够更加稳定的在自己不知道的情况下稳定调用Function calling进行搜索和问答。但是，限于GPT-4模型的8k最大上下文对话长度，很多时候为了输入更多的相关信息从而获得更好的搜索结果，我们往往会倾向于选取GPT-3.5-16k模型，因此，为了解决GPT-3.5模型的“模型不清楚自己不知道”这一问题，我们可以按照本节开始时介绍的思路，在输入问题的时增加一层判断，即引导大模型判断这个问题是否知道，然后在确定模型不知道的情况下再进行搜索。但是有趣的是，GPT-3.5模型幻觉问题“根深蒂固”，有时候我们甚至无法通过再system message中输入“如果知道问题的答案，请回答问题答案，如果不知道问题答案，请回复‘抱歉，这个问题我并不知道’”这种提示来让模型认识到自己不知道：

In [31]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        messages=[
            {"role": "system", "content": "根据用户输入的问题进行回答，如果知道问题的答案，请回答问题答案，如果不知道问题答案，请回复‘抱歉，这个问题我并不知道’"},
            {"role": "user", "content": "请问GPT-3.5微调总共有几个阶段？"}
        ],
        functions=functions,
        function_call="auto",  
    )
response["choices"][0]["message"]["content"]

'GPT-3.5微调总共有两个阶段：预训练和微调。在预训练阶段，GPT-3.5模型使用大规模的文本数据进行自监督学习，学习语言模型的能力。然后，在微调阶段，使用特定领域的数据对预训练的模型进行进一步训练，以适应特定任务或应用场景。'

而如果换一种提示，即提示模型判断这个问题是否超出自己的知识库范围，此时模型会变得“老实起来”，能够实事求是的判断问题是否超出自己的知识库范围：

In [33]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        messages=[
            {"role": "system", "content": "根据用户输入的问题进行回答，如果知道问题的答案，请回答问题答案，如果不知道问题答案，请回复‘抱歉，这个问题我并不知道’"},
            {"role": "user", "content": "请问“GPT-3.5微调总共有几个阶段”这个问题是否超出你的知识库范围？"}
        ],
        functions=functions,
        function_call="auto",  
    )
response["choices"][0]["message"]["content"]

'是的，这个问题超出了我的知识库范围。抱歉，我无法回答这个问题。'

因此，我们完全可以采用“是否超出知识库”这一提示方法，判断当前问题是否需要先进行搜索然后再进行回答，即我们需要先创建一个判别大模型，对问题能否回答进行判别，然后再根据判别结果来调用大模型进行回答、或者先进行搜索再进行回答。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/71c2f15cbe4c470594024d8b4edb525.png" alt="71c2f15cbe4c470594024d8b4edb525" style="zoom:33%;" />

- 两层模型之间的稳定加密通信

&emsp;&emsp;而根据此前的多层大模型架构经验，要将前一层大模型输出的结果信息稳定的传输给下一层（即让下一层模型清楚的了解到这个问题是该调用模型回答还是应该调用搜索），其实并不是一个简单的方法。在此前的项目中，我们采用的方法是尽量规定上一层模型的输出内，例如当不知道时回答不知道，然后在下一层再匹配上一层输出结果中是否包含“不知道”这一关键词，然后再进行判断。但这种方法在当前项目的应用中存在一定的问题，那就是GPT-3.5模型对于一些“规定”理解的并不深刻，并不一定会按照人工要求进行输出，例如我们规定超出知识库范围则回答“不知道”，模型有可能回答“不清楚”、“超出了知识库，非常抱歉”等内容，并且有些问题的正确答案中就包含“不知道”3个字，这些都会导致简单的匹配“不知道”3个字可能会出现误判。因此，一种非常好的解决问题的方法是在超出模型知识库范畴时，让模型输出一段毫无意义的“字符串密钥”，由于该信息过于特殊，一方面模型找不到类似的可替换的同义字符，另一方面这些非常特殊的字符也不太可能出现在正确答案中，因此可以作为下一层模型识别上一层模型输出结果的关键字符：

In [57]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        messages=[
            {"role": "user", "content": "请问“请问“GPT-3.5微调总共有几个阶段”这个问题是否超出你的知识库范围？若这个超出知识库范围，请回答：“不知道”"}
        ]
    )
response["choices"][0]["message"]["content"]

'抱歉，这个问题超出了我的知识库范围，我无法做出回答。'

接下来采用一段30个字符的随机密钥作为“密语”：

In [48]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        messages=[
            {"role": "user", "content": "请问“GPT-3.5微调总共有几个阶段”这个问题是否超出你的知识库范围？若这个超出知识库范围，请回答：“ioHEw9ZGjxETRV2h97CUHxZMLqdzgr”"}
        ]
    )
response["choices"][0]["message"]["content"]

'ioHEw9ZGjxETRV2h97CUHxZMLqdzgr'

能够发现，通过密钥作为“判别符”能够大幅提升上下层信息传递的准确率。而对于下一层模型来说，我们可以以此作为判断：即当上一层模型结果输出内容包含'ioHEw9ZGjxETRV2h97CUHxZMLqdzgr'时，则说明接下来的回答需要进行搜索。

&emsp;&emsp;同时，为了更加安全的进行加密，我们可以创建如下函数专门用于创建一个长度为30字符的无意义字符串密钥：

In [69]:
import random
import string

def generate_random_key(length=30):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

random_key = generate_random_key()
print(random_key)

41WQdO2KgbxUXhvpKbV7stpLfOvW2x


而在后续每次调用模型时，都可以临时创建一个随机字符串，作为本次通信的“凭证”。

- 创建判别模型

&emsp;&emsp;接下来即可据此创建一个判别模型，即引导大模型先判断当前问题是否超出了知识库范围，如果超出了范围，则输出True，若没有超出知识库范围，则正常进行回答即可。具体函数创建过程如下：

In [221]:
def identify_model(q):
    
    # 创建密钥
    sk = generate_random_key()
    
    # 调用模型进行判别
    response = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=[
                {"role": "system", "content": "你是一个识别器，专门用于判别用户问题是否超出了你的当前知识库范围。\
                若超出知识库范围，请回答“%s”，若未超出知识库范围，请正常回答" % sk},
                {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
                {"role": "assistant", "content": "%s" % sk},
                {"role": "user", "content": q}
            ]
        )
    res = response["choices"][0]["message"]["content"]
    
    if sk in res or '对不起' in res or '抱歉' in res or '超出知识库' in res:
        return(True)
    else:
        return(res)

In [176]:
q = '请问什么是机器学习？'
identify_model(q)

'机器学习是人工智能的一个分支，它的目标是开发和应用算法让计算机可以学习从数据中获取的模式。在一般情况下，机器学习算法可以根据已有数据预测出新的结果。基于获取的数据，这些算法会自我调整并生成新模型来预测未来结果。这个过程不需要人工进行明确的编程，因为系统是通过学习数据自动调整的。'

In [190]:
q = '请帮我编写GPT-3.5微调代码'
identify_model(q)

True

In [189]:
q = '2023年推出的ChatGPT企业版有哪些不一样的功能？'
identify_model(q)

True

- 用户问题转化为知乎搜索关键词

&emsp;&emsp;而在具体执行搜索时，需要注意的是，用户提出的问题并不一定是最适合在知乎上进行搜索的关键词，例如用户提问“我想要知道“GPT-3.5微调总共需要几步”，此时应该将其转化为更加准确凝练的关键词并输入到知乎上进行搜索，方可提高搜索结果准确性。这里我们也完全可以让大模型自主将用户提的问题转化为适合在知乎上进行搜索的搜索关键词。当然，为达此目的，我们需要首先测试大模型是否知道知乎，以及大模型是否知道如何设计搜索关键词：

In [58]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
            {"role": "user", "content": "你知道什么是知乎么？"}
        ]
    )
response["choices"][0]["message"]["content"]

'知乎是中国最大的知识分享和自我提升社区，是一个连接各行各业专业人士的知识分享平台。用户可以在这里提问、回答问题、学习和分享各类专业知识和独到见解。知乎的宗旨是“让人人平等地分享知识、经验和见解，让世界更好地理解并思考。”'

In [59]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
            {"role": "user", "content": "我如果想在知乎上搜索一些问题的答案，在设计搜索关键词方面，有什么技巧么？"}
        ]
    )
response["choices"][0]["message"]["content"]

'1. 尽量具体：尽可能详细地描述你的问题，搜索结果将更准确。\n\n2. 使用了解的术语：如果你在咨询一些专业问题，使用该领域的专业术语将有助于你找到更专业的答案。\n\n3. 使用引号：将你想要搜索的词组或句子放在引号中，搜索结果将完全匹配你的查询。\n\n4. 使用“-”符号：如果你想要排除某个词，可以在该词前加上“-”符号，例如搜索“电视 -海信”将显示包含“电视”但不包含“海信”的搜索结果。\n\n5. 使用通配符：你也可以使用“*”来代表任何你不确定的词，例如搜索“最好的*手机”将显示任何涉及“最好的_ _手机”的文章。\n\n6. 使用“OR”：如果你想同时搜索两个词，可以使用“OR”，例如搜索“iphone OR android”，结果将显示包含这两个词中任何一个的内容。\n\n7. 互相关联的词汇：使用与搜索主题相关联的词汇进行搜索，有时能够得到不一样的想法和回答。\n\n8. 分步查询：根据你得到的结果，逐步改进或更改你的搜索词，而不是一开始就尝试完成完全正确的查询。\n\n9. 如果知道回答者的信息也可以在关键词中加入，如"xx老师的观点"，或者"xx公司的策略"等等。\n\n这只是一些基本的搜索技巧，实际上你可以通过阅读帮助中心或参与论坛讨论来了解更多的搜索技巧。'

然后测试让大模型将用户问题转化为搜索关键词：

In [62]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
            {"role": "user", "content": "例如，我想要知道“GPT-3.5微调总共需要几步”，\
            现在围绕这个问题在知乎上进行搜索，应该怎样设计这个问题的搜索关键词呢？"}
        ]
    )
response["choices"][0]["message"]["content"]

'"GPT-3.5微调步骤" 或者 "GPT-3.5微调流程"'

接下来，我们即可将这个过程封装为一个完整的函数，专门使用GPT-4来将用户问题转化为适合进行搜索的关键词：

In [64]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
             {"role": "system", "content": "你专门负责将用户的问题转化为知乎网站搜索关键词，只返回一个你认为最合适的搜索关键词即可"},
             {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
             {"role": "assistant", "content": "GPT-3.5微调流程"},
             {"role": "user", "content": "请问ChatGPT企业版都有哪些功能？"}
        ]
    )
response["choices"][0]["message"]["content"]

'ChatGPT企业版功能'

In [66]:
def convert_keyword(q):
    """
    将用户输入的问题转化为适合在知乎上进行搜索的关键词
    """
    response = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=[
                 {"role": "system", "content": "你专门负责将用户的问题转化为知乎网站搜索关键词，只返回一个你认为最合适的搜索关键词即可"},
                 {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
                 {"role": "assistant", "content": "GPT-3.5微调流程"},
                 {"role": "user", "content": q}
            ]
        )
    q = response["choices"][0]["message"]["content"]
    return q

然后我们需要将这个转化函数嵌入get_answer中来进行执行，令其自动进行搜索词优化。同时为了更加明确的展示当前搜索和问答的进度，我们在get_answer中额外添加一些打印信息，具体函数修改结果如下：

In [191]:
def get_answer(q):
    """
    当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 调用转化函数，将用户的问题转化为更适合在知乎上进行搜索的关键词
    q = convert_keyword(q)
    
    # 默认搜索返回10个答案
    print('正在接入谷歌搜索，查找和问题相关的答案...')
    results = google_search(query=q, num_results=10, site_url='https://zhihu.com/')
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    print('正在读取搜索的到的相关答案...')
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    print('正在进行最后的整理...')
    return(content)

- Function calling中手动调用外部函数方法

&emsp;&emsp;而既然是依据“密钥”来判断是否调用搜索功能，那么该外部函数的调用就不能再使用此前定义的auto function calling功能了，而是需要我们手动来调用这个外部函数（即接收到密钥时一定要调用外部函数）。而根据Ch.8相关内容介绍，当我们在Chat模型的function_call参数位置上输入对应需要调用的外部函数名称时，模型就不再会自主判断是否需要调用外部函数，而是一定会通过调用该函数来回答原始问题，这个过程也就相当于是手动调用外部函数。具体实现过程如下：

In [95]:
get_answer

<function __main__.get_answer(q)>

In [96]:
functions_list = [get_answer]

In [97]:
functions = auto_functions(functions_list)
functions

[{'name': 'get_answer',
  'description': '当你无法回答某个问题时，调用该函数，能够获得答案',
  'parameters': {'type': 'object',
   'properties': {'q': {'type': 'string', 'description': '询问的问题'}},
   'required': ['q']}}]

In [98]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        messages=[
            {"role": "user", "content": "请问GPT-3.5微调总共有几个阶段？"}
        ],
        functions=functions,
        function_call={"name": "get_answer"},  
    )
response["choices"][0]["message"]

<OpenAIObject at 0x202e5d876a0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_answer",
    "arguments": "{\n  \"q\": \"GPT-3.5\u5fae\u8c03\u6709\u51e0\u4e2a\u9636\u6bb5\"\n}"
  }
}

考虑到之后我们需要使用run_conversation自动化执行Function calling功能，因此这里我们需要围绕run_conversation稍作调整，使其支持手动实现Function calling功能。相关修改的地方并不多，只要添加一个function_call参数即可，同时在first response中修改function_call=function_call即可。修改之后的run_conversation如下：

In [94]:
def run_conversation(messages, functions_list=None, model="gpt-4-0613", function_call="auto"):
    """
    能够自动执行外部函数调用的Chat对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为gpt-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = openai.ChatCompletion.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response["choices"][0]["message"]
        final_response = response_message["content"]
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        functions = auto_functions(functions_list)
        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # first response
        response = openai.ChatCompletion.create(
                        model=model,
                        messages=messages,
                        functions=functions,
                        function_call=function_call)
        response_message = response["choices"][0]["message"]

        # 判断返回结果是否存在function_call，即判断是否需要调用外部函数来回答问题
        if response_message.get("function_call"):
            # 需要调用外部函数
            # 获取函数名
            function_name = response_message["function_call"]["name"]
            # 获取函数对象
            fuction_to_call = available_functions[function_name]
            # 获取函数参数
            function_args = json.loads(response_message["function_call"]["arguments"])
            # 将函数参数输入到函数中，获取函数计算结果
            function_response = fuction_to_call(**function_args)

            # messages中拼接first response消息
            messages.append(response_message)  
            # messages中拼接函数输出结果
            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                }
            )  
            # 第二次调用模型
            second_response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
            )  
            # 获取最终结果
            final_response = second_response["choices"][0]["message"]["content"]
        else:
            final_response = response_message["content"]
            
    del messages
    
    return final_response

#### 3.优化后的自动搜索问答机器人

&emsp;&emsp;在具体介绍了一列关于搜索问答机器人的优化方法之后，接下来我们尝试将这一系列优化方法融合到一个流程中，并编写相应函数来完整实现优化后的自动搜索问答机器人。

- 优化后的自动搜索问答机器人完整流程

&emsp;&emsp;首先，根据此前的介绍，我们整理优化后的自动搜索问答机器人完整实现流程如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/%E8%87%AA%E5%8A%A8%E6%90%9C%E7%B4%A2%E9%97%AE%E7%AD%94%E6%9C%BA%E5%99%A8%E4%BA%BA%E5%AE%9E%E7%8E%B0%E6%B5%81%E7%A8%8B.jpeg" alt="自动搜索问答机器人实现流程" style="zoom:33%;" />

- 优化后的自动搜索问答机器人函数编写

&emsp;&emsp;在进行了一些列函数储备之后，接下来的自动搜索问答机器人的实际代码执行流程就变得非常清晰了，具体执行过程如下：

In [210]:
def auto_search_answer(q):
    # 调用判别模型
    res = identify_model(q)
    if res == True:
        messages = [{"role": "user", "content": q}]
        res =run_conversation(messages=messages, 
                              functions_list=[get_answer], 
                              model="gpt-3.5-turbo-16k-0613", 
                              function_call={"name": "get_answer"})
    return(res)

- 优化后的自动搜索问答机器人性能测试

然后我们测试函数功能：

In [214]:
auto_search_answer('请问最新ChatGPT企业版有哪些功能？')

正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...


'最新的ChatGPT企业版具有以下功能：\n\n1. 企业级安全性和隐私性：ChatGPT企业版提供数据加密、符合SOC 2标准，以保护企业数据的安全性和隐私性。\n\n2. 无限制的GPT-4访问：ChatGPT企业版可以无限制地访问GPT-4，并且速度比之前的版本提高了两倍。\n\n3. 更长的上下文窗口：ChatGPT企业版支持32k Tokens上下文窗口，比之前的版本提供了更长的上下文处理能力。\n\n4. 高级数据分析功能：ChatGPT企业版提供对高级数据分析（先前称为代码解释器）的无限制访问，可以帮助技术和非技术团队在几秒钟内分析信息。\n\n5. 自定义选项：企业可以定制ChatGPT并连接自身数据，将ChatGPT与其他应用程序集成，以扩展其知识和能力。\n\n除了以上功能，OpenAI还计划推出更多功能，包括可自定义的程序连接、适用于小型团队的ChatGPT Business版本、更强大的高级数据分析和针对特定职能的工具等。'

通过查阅本地文件夹，能够看到该问题的搜索结果如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/44ce767953817be18ddff7882f8d08a.png" alt="44ce767953817be18ddff7882f8d08a" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/4d7eec6a3b3c626a5763e765be3a15a.png" alt="4d7eec6a3b3c626a5763e765be3a15a" style="zoom:33%;" />

紧接着测试下一个问题：

In [216]:
auto_search_answer('请帮我介绍下P-tuning v2微调算法')

正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...


'P-tuning v2微调算法是一种参数高效的微调方法，可以使预训练语言模型 (PLM) 高效适应各种下游应用，而无需微调模型的所有参数。与传统的微调方法相比，P-tuning v2仅微调少量（额外）模型参数，从而大大降低了计算和存储成本。最近的研究表明，P-tuning v2的性能可以与完全微调相媲美。\n\nP-tuning v2的核心思想是在预训练模型中插入可微调的标记，以指导模型对不同任务的微调。与传统的微调方法不同，P-tuning v2不仅仅在输入层进行微调，还在模型的不同层插入连续的可微调token，从而增加微调的灵活性和效果。\n\n具体而言，P-tuning v2的步骤如下：\n1. 在输入序列前面插入一个连续的可微调token，以指导模型进行下游任务的微调。\n2. 在预训练模型的每个层中，插入可微调的token，并使用这些token来微调模型的参数。\n3. 在微调过程中，只微调新增的参数，将预训练模型的其他参数固定不变，从而减少微调的参数量。\n4. 根据下游任务的需求，可以自定义可微调token的长度和类型，以适应不同的任务要求。\n\n通过以上步骤，P-tuning v2能够在保持预训练模型知识的基础上，针对特定任务进行高效微调。这种方法不仅在参数效率上有所突破，还在各种下游任务上展现出良好的性能。\n\n需要注意的是，P-tuning v2仍然面临一些挑战，比如参数灾难性遗忘和过度拟合等问题。为了缓解这些问题，可以将微调语料与通用学习语料一起使用，并在微调过程中控制可微调token的拟合程度。\n\n总而言之，P-tuning v2是一种针对大型语言模型的参数高效微调方法，能够在保证性能的同时降低计算和存储成本，具有很大的应用潜力。'

该问题的搜索结果本地保存内容如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1c418471df18194dfc86c4fb8ea5f2e.png" alt="1c418471df18194dfc86c4fb8ea5f2e" style="zoom:33%;" />

<center>

<img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/d83b6f3f7f5dc73a946edc46c0008a8.png" alt="d83b6f3f7f5dc73a946edc46c0008a8" style="zoom:33%;" />

继续进行测试：

In [217]:
auto_search_answer('请帮我详细介绍下P-tuning v2微调的代码执行流程')

正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...


'P-tuning v2微调的代码执行流程如下：\n\n1. 数据准备：首先需要准备训练数据和验证数据。训练数据是用于微调的数据集，验证数据用于评估模型在不同训练阶段的性能。\n\n2. 模型选择：选择基础的预训练语言模型作为基础模型。可以选择一些流行的预训练模型，如RoBERTa、GPT等。\n\n3. 模型初始化：根据选择的预训练模型，加载对应的模型权重和配置。\n\n4. 建立微调模型：根据基础模型的配置，构建微调模型。可以使用适配器、前缀调整等方法进行模型微调。\n\n5. 定义训练循环：定义训练循环，包括前向传播、损失计算、反向传播和参数更新等步骤。可以使用优化器来更新参数。\n\n6. 执行微调：在训练循环中，使用训练数据进行模型微调。根据训练数据的特点和任务需求，设置合适的训练参数，如学习率、批量大小等。\n\n7. 评估模型性能：在每个训练阶段，使用验证数据评估模型在验证集上的性能。可以计算准确率、损失值等指标来评估模型的效果。\n\n8. 调整超参数：根据模型在验证集上的性能表现，调整超参数，如学习率、正则化权重等。可以使用学习率衰减、提前停止等策略来优化模型。\n\n9. 完成微调：重复执行训练和验证的过程，直到模型在验证集上达到满意的性能。\n\n10. 保存模型：在微调完成后，保存微调后的模型权重和配置，以备后续使用。\n\n11. 模型应用：将微调后的模型应用于具体的下游任务，如文本分类、机器翻译等。\n\n需要注意的是，P-tuning v2微调的代码实现流程可能会因具体的任务和框架而有所差异。上述流程仅为一般的参考，具体的实现细节可以根据具体情况进行调整和优化。'

该问题的搜索结果本地保存内容如下：

<img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/970ec1b2ee258efaf209bba8a03f5fc.png" alt="970ec1b2ee258efaf209bba8a03f5fc" style="zoom:33%;" />

<img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/d728f311e911fb8b2fefff826a2f28d.png" alt="d728f311e911fb8b2fefff826a2f28d" style="zoom:33%;" />

再测试最后一个问题：

In [220]:
auto_search_answer('相比其他微调框架，P-tuning v2微调有什么优势呢？')

正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...


'P-tuning v2微调相比其他微调框架具有以下优势：\n\n1. 参数高效：P-tuning v2只微调少量（额外）模型参数，大大降低了计算和存储成本。它通过灵活的参数适配和组合技术，可以在适应不同任务时节省大量的参数和计算资源。\n\n2. 通用性：P-tuning v2在各种规模和任务上都表现出色，无论是小型模型还是大型模型，都能取得可比甚至更好的性能。因此，它是一种通用的微调方法，适用于不同的应用场景。\n\n3. 多任务学习：P-tuning v2引入了多任务学习，先在多任务的prompt上进行预训练，然后再适配下游任务。这种方法可以充分利用不同任务之间的相似性和互补性，提高模型的泛化能力和适应性。\n\n4. 灵活性：P-tuning v2通过引入可训练的prompt部分，可以根据不同任务的需求进行调整和修改。它不仅在模型结构上具有灵活性，还在预训练和微调过程中提供了更多的自定义选项。\n\n总之，P-tuning v2是一种高效、通用、灵活的微调框架，可以在节省计算资源的同时获得良好的性能。它在参数高效性和任务适应性上具有优势，是当前研究领域的重要进展之一。'

该问题的搜索结果本地保存内容如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/43cf0d2b194885e6486b07b756b46ff.png" alt="43cf0d2b194885e6486b07b756b46ff" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/52650fb768c8b44e0bd87c280cd0aae.png" alt="52650fb768c8b44e0bd87c280cd0aae" style="zoom:33%;" />

能够看出，整体搜索得到的内容质量还是非常高的，并且从问答模型的答案质量非常高（相比GPT-3.5模型而言）。至此优化之后的搜索问答机器人便完整创建完毕，之后我们再实际课程中，也将经常使用该机器人用于辅助问题回答。

---

#### <center>自动搜索问答项目代码总结

&emsp;&emsp;考虑到本项目单独创建的函数众多，且部分函数经过了多次修改，这里统一将本项目中所涉及的代码单独再次进行定义并在本地保存为autoSearch.py文件，方便后续直接调用。

In [101]:
def google_search(query, num_results=10, site_url=None):
    
    api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
    cse_id = os.getenv("CSE_ID")
    
    url = "https://www.googleapis.com/customsearch/v1"

    # API 请求参数
    if site_url == None:
        params = {
        'q': query,          
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results   
        }
    else:
        params = {
        'q': query,         
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results,  
        'siteSearch': site_url
        }

    # 发送请求
    response = requests.get(url, params=params)
    response.raise_for_status()

    # 解析响应
    search_results = response.json().get('items', [])

    # 提取所需信息
    results = [{
        'title': item['title'],
        'link': item['link'],
        # 'snippet': item['snippet']
    } for item in search_results]

    return results

In [3]:
def windows_compatible_name(s, max_length=255):
    """
    将字符串转化为符合Windows文件/文件夹命名规范的名称。
    
    参数:
    - s (str): 输入的字符串。
    - max_length (int): 输出字符串的最大长度，默认为255。
    
    返回:
    - str: 一个可以安全用作Windows文件/文件夹名称的字符串。
    """

    # Windows文件/文件夹名称中不允许的字符列表
    forbidden_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']

    # 使用下划线替换不允许的字符
    for char in forbidden_chars:
        s = s.replace(char, '_')

    # 删除尾部的空格或点
    s = s.rstrip(' .')

    # 检查是否存在以下不允许被用于文档名称的关键词，如果有的话则替换为下划线
    reserved_names = ["CON", "PRN", "AUX", "NUL", "COM1", "COM2", "COM3", "COM4", "COM5", "COM6", "COM7", "COM8", "COM9", 
                      "LPT1", "LPT2", "LPT3", "LPT4", "LPT5", "LPT6", "LPT7", "LPT8", "LPT9"]
    if s.upper() in reserved_names:
        s += '_'

    # 如果字符串过长，进行截断
    if len(s) > max_length:
        s = s[:max_length]

    return s

In [4]:
def get_search_text(q, url):

    cookie = os.getenv('search_cookie')
    user_agent = os.getenv('search_user_agent')    
    title = None
    
    code_ = False
    headers = {
        'authority': 'www.zhihu.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': cookie,
        'upgrade-insecure-requests': '1',
        'user-agent':user_agent,
    }

    # 普通问答地址
    if 'zhihu.com/question' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div/div/div/div[2]/div/div[2]/div/div/div[2]/span[1]/div/div/span/p/text()')
    
    # 专栏地址
    elif 'zhuanlan' in url:
        headers['authority'] = 'zhaunlan.zhihu.com'
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div[1]/div/main/div/article/header/h1/text()')[0]
        text_d = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div/p/text()')
        code_ = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div//pre/code/text()')  
            
    # 特定回答的问答网址
    elif 'answer' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div[1]/div/div[3]/div/div/div/div[2]/span[1]/div/div/span/p/text()')

    if title == None:
        return None
    
    else:
        title = windows_compatible_name(title)

        # 创建问题答案正文
        text = ''
        for t in text_d:
            txt = str(t).replace('\n', ' ')
            text += txt

        # 如果有code，则将code追加到正文的追后面
        if code_:
            for c in code_:
                co = str(c).replace('\n', ' ')    
                text += co

        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")     
        json_data = [
            {
                "link": url,
                "title": title,
                "content": text,
                "tokens": len(encoding.encode(text))
            }
        ]

        with open('./auto_search/%s/%s.json' % (q, title), 'w') as f:
            json.dump(json_data, f)

        return title

In [5]:
def generate_random_key(length=30):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

In [6]:
def identify_model(q):
    
    # 创建密钥
    sk = generate_random_key()
    
    # 调用模型进行判别
    response = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=[
                {"role": "system", "content": "你是一个识别器，专门用于判别用户问题是否超出了你的当前知识库范围。\
                若超出知识库范围，请回答“%s”，若未超出知识库范围，请正常回答" % sk},
                {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
                {"role": "assistant", "content": "%s" % sk},
                {"role": "user", "content": q}
            ]
        )
    res = response["choices"][0]["message"]["content"]
    
    if sk in res or '对不起' in res or '抱歉' in res or '超出知识库' in res:
        return(True)
    else:
        return(res)

In [7]:
def convert_keyword(q):
    """
    将用户输入的问题转化为适合在知乎上进行搜索的关键词
    """
    response = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=[
                 {"role": "system", "content": "你专门负责将用户的问题转化为知乎网站搜索关键词，只返回一个你认为最合适的搜索关键词即可"},
                 {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
                 {"role": "assistant", "content": "GPT-3.5微调流程"},
                 {"role": "user", "content": q}
            ]
        )
    q = response["choices"][0]["message"]["content"]
    return q

In [8]:
def get_answer(q):
    """
    当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 调用转化函数，将用户的问题转化为更适合在知乎上进行搜索的关键词
    q = convert_keyword(q)
    
    # 默认搜索返回10个答案
    print('正在接入谷歌搜索，查找和问题相关的答案...')
    results = google_search(query=q, num_results=10, site_url='https://zhihu.com/')
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    print('正在读取搜索的到的相关答案...')
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    print('正在进行最后的整理...')
    return(content)

In [9]:
def run_conversation(messages, functions_list=None, model="gpt-4-0613", function_call="auto"):
    """
    能够自动执行外部函数调用的Chat对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为gpt-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = openai.ChatCompletion.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response["choices"][0]["message"]
        final_response = response_message["content"]
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        functions = auto_functions(functions_list)
        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # first response
        response = openai.ChatCompletion.create(
                        model=model,
                        messages=messages,
                        functions=functions,
                        function_call=function_call)
        response_message = response["choices"][0]["message"]

        # 判断返回结果是否存在function_call，即判断是否需要调用外部函数来回答问题
        if response_message.get("function_call"):
            # 需要调用外部函数
            # 获取函数名
            function_name = response_message["function_call"]["name"]
            # 获取函数对象
            fuction_to_call = available_functions[function_name]
            # 获取函数参数
            function_args = json.loads(response_message["function_call"]["arguments"])
            # 将函数参数输入到函数中，获取函数计算结果
            function_response = fuction_to_call(**function_args)

            # messages中拼接first response消息
            messages.append(response_message)  
            # messages中拼接函数输出结果
            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                }
            )  
            # 第二次调用模型
            second_response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
            )  
            # 获取最终结果
            final_response = second_response["choices"][0]["message"]["content"]
        else:
            final_response = response_message["content"]
            
    del messages
    
    return final_response

In [10]:
def auto_search_answer(q):
    # 调用判别模型
    res = identify_model(q)
    if res == True:
        messages = [{"role": "user", "content": q}]
        res =run_conversation(messages=messages, 
                              functions_list=[get_answer], 
                              model="gpt-3.5-turbo-16k-0613", 
                              function_call={"name": "get_answer"})
    return(res)

---

### 五、自动搜索问答流程拓展实战

&emsp;&emsp;对于自动搜索问答机器人的应用场景，除了应用于知乎的技术问答贴的相关内容搜索问答外，上述项目的流程也可以应用于其他很多实战场景中。考虑到学员有很多需求都是学习方面需求，接下来我们将上述优化后的自动搜索问答流程应用于对Github和Hugging face的相关技术内容搜索中，相信对于大多数大模型技术人员来说，Github和Hugging face都是必不可少的获取关键技术信息的网站，而通过介绍将这两个网站接入GPT的方法，一方面深化本小节介绍的搜索问答流程的技术应用理解，另一方面也为大家后续学习提供更多智能化问答工具。

#### 1.围绕Github的智能搜索问答流程

&emsp;&emsp;首先我们考虑将自动搜索问答流程应用于对Github的相关内容搜索过程中。由于Github本身提供了非常多API接口，能够直接通过调用API来获取相关项目信息，因此整个自动搜索问答流程可以免去爬虫过程，整体来看将Github接入GPT的过程并不会特别复杂。

- Github智能问答机器人项目目标与实现思路

&emsp;&emsp;首先，既然是围绕Github上的开源项目进行智能问答，我们问答的核心目的就在于询问和了解一些开源项目梗概，同时查询还有哪些相关且较为流行的项目。当然使用此前获取的谷歌搜索API在Github上进行搜索肯定是可行的，但需要注意的是，在Github上的搜索和在知乎上进行搜索肯定是有去别的，对于Github来说，我们需要围绕每个问题提炼问题背后的项目名称，再围绕某个项目进行搜索。例如当我们搜索“请帮我介绍下DeepSpeed”时，需要先提炼这个问题背后的项目名称，再在Github上进行搜索，因此这里首先需要调整convert_keyword函数。其次，我们还需要简单了解Github上的网站规则，希望谷歌搜索得到的链接都是项目链接而不是其他讨论贴的链接；其三，考虑到最终我们了解每个开源项目的途径是查阅其readme文档，因此我们也需要简单了解Github API使用方法，并通过API调取对应项目的readme进行查阅。

&emsp;&emsp;除此之外，其他流程和此前的搜索问答执行流程类似。接下来我们就按部就班尝试将Github接入GPT模型中。

##### 1.1 convert_keyword函数修改

&emsp;&emsp;根据此前介绍，我们需要单独编写一个适用于将用户问题转化为Github搜索关键词、也就是项目名称的搜索关键词转化函数。当然这个函数的编写过程也并不复杂，我们只需要在此前定义的convert_keyword函数基础上稍作修改即可：

In [2]:
def convert_keyword_github(q):
    """
    将用户输入的问题转化为适合在Github上进行搜索的关键词
    """
    response = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=[
                 {"role": "system", "content": "你专门负责将用户的问题转化为Github上的搜索关键词，只返回一个你认为最合适的搜索关键词即可"},
                 {"role": "user", "content": "请问DeepSpeed是什么？"},
                 {"role": "assistant", "content": "DeepSpeed"},
                 {"role": "user", "content": q}
            ]
        )
    q = response["choices"][0]["message"]["content"]
    return q

In [98]:
convert_keyword_git('请帮我介绍下p-tunning v2')

'p-tunning v2'

##### 1.2 Github搜索规则与搜索范围调整

&emsp;&emsp;接下来，我们进入到搜索过程，围绕给定的某个搜索关键词，先观察google_search在Github上的搜索结果：

In [11]:
google_search(query='p-tunning v2', num_results=10, site_url='https://github.com/')

[{'title': 'THUDM/P-tuning-v2: An optimized deep prompt tuning ... - GitHub',
  'link': 'https://github.com/THUDM/P-tuning-v2',
  'snippet': 'An optimized deep prompt tuning strategy comparable to fine-tuning across scales and tasks - GitHub - THUDM/P-tuning-v2: An optimized deep prompt tuning\xa0...'},
 {'title': 'THUDM/P-tuning: A novel method to tune language models ... - GitHub',
  'link': 'https://github.com/THUDM/P-tuning',
  'snippet': "[2021-10-15] P-tuning v2 is out! Check our Github repo. A novel method to tune language models. Codes and datasets for paper ``GPT understands, too''. Xiao\xa0..."},
 {'title': 'What parameters is really trained during P-tuning v2? · Issue #1 ...',
  'link': 'https://github.com/THUDM/P-tuning-v2/issues/1',
  'snippet': "Oct 15, 2021 ... Could you please clarify what parameters are really tuned during p-tuning? Is it MLP producing input embeddings for a large model? Or maybe it's\xa0..."},
 {'title': 'p-tuning-v2 · GitHub Topics · GitHub',
  'link

能够发现，搜索得到的第一个链接就是p-tunning v2项目地址，而后还包括一些与之相关的项目，如p-tunning、hugging face的PEFT项目，以及一个ChatGLM在p-tunning v2上微调的项目（最后一个链接、同时还包括很多其他微调方法实现），此外，搜索得到的链接中还包括一些issues页面和一些topics页面，总的来看整体搜索结果质量很高。

&emsp;&emsp;不过需要注意的是，在上述搜索得到的页面中，只有项目主页面才有完整的readme说明文档，而这也是之后输入给GPT模型的核心信息，而其他页面，如issues页面、topics页面以及blob页面（某特定文件的页面）则无法提供整段的高质量文档。因此这些链接需要被排除在外。根据此前的介绍，topics页面是一类最容易被排除在搜索结果之外的页面，我们只需要将github.com/topic 设置为“要排除的网站”即可。和此前一样，这个设置过程需要在可编程引擎主页完成。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/dbf5e87f3cefe0e9d1e12ccfaea7130.png" alt="dbf5e87f3cefe0e9d1e12ccfaea7130" style="zoom:33%;" />

然后再之后的搜索结果中，就不会出现topic页面了。而对于issues页面和blob页面，则会在后续代码环节中被剔除。

In [73]:
search_results = google_search(query='p-tunning v2', num_results=10, site_url='https://github.com/')
search_results

[{'title': 'THUDM/P-tuning-v2: An optimized deep prompt tuning ... - GitHub',
  'link': 'https://github.com/THUDM/P-tuning-v2',
  'snippet': 'An optimized deep prompt tuning strategy comparable to fine-tuning across scales and tasks - GitHub - THUDM/P-tuning-v2: An optimized deep prompt tuning\xa0...'},
 {'title': 'THUDM/P-tuning: A novel method to tune language models ... - GitHub',
  'link': 'https://github.com/THUDM/P-tuning',
  'snippet': "[2021-10-15] P-tuning v2 is out! Check our Github repo. A novel method to tune language models. Codes and datasets for paper ``GPT understands, too''. Xiao\xa0..."},
 {'title': 'What parameters is really trained during P-tuning v2? · Issue #1 ...',
  'link': 'https://github.com/THUDM/P-tuning-v2/issues/1',
  'snippet': "Oct 15, 2021 ... Could you please clarify what parameters are really tuned during p-tuning? Is it MLP producing input embeddings for a large model? Or maybe it's\xa0..."},
 {'title': 'huggingface/peft: PEFT: State-of-the-art Param

##### 1.3 Github API使用与项目readme文档获取

- Github token获取

&emsp;&emsp;在获取了相关项目主页链接之后，接下来我们则需要考虑如何获取这些项目的readme文档。这里我们不再需要使用爬虫，而是简单调用Github API即可获取完整的项目readme文档。而要使用Github API，则首先需要获取Github token。Github token全称为Github Personal Access Token，用于在调用Github时进行身份严重，其本质上就是一串字符串形式的加密密钥，也就相当于OpenAI API Key。

&emsp;&emsp;而Github token的获取也非常简单，只需要拥有一个Github账号即可免费获取。在登录Github之后，我们可以在个人账户的设置页面进行token获取：https://github.com/settings/tokens

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/ec90bc77684124e5076821f7731a427.png" alt="ec90bc77684124e5076821f7731a427" style="zoom:33%;" />

这里点击Gererate new token即可进入到token创建页面，这里我们随意输入Note即可，然后设置token生效的期限。若该token只是为了查询readme文档，则无需勾选下面的任何选项（无需赋予token额外的权限）：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/9382384d975db1f3623c35fe3f0cea4.png" alt="9382384d975db1f3623c35fe3f0cea4" style="zoom:33%;" />

然后直接点击Generate token即可。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/86554f98c0fe91894c15be32932c5d8.png" alt="86554f98c0fe91894c15be32932c5d8" style="zoom: 33%;" />

然后在弹出的页面复制token字符串。然后同样以系统环境变量的方式进行保存和后续的读取：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/e78fdd369e8735d0a417f32e9aee8f2.png" alt="e78fdd369e8735d0a417f32e9aee8f2" style="zoom:33%;" />

In [74]:
github_token = os.getenv('GITHUB_TOKEN')

而在获取了Github token之后，接下来即可使用requests来获取对应项目的readme内容了。

- 借助Github API获取readme文档

&emsp;&emsp;Github API获取readme文档同样是借助request过程来实现，具体实现流程如下：

In [79]:
import requests
import base64

owner = "THUDM"
repo = "P-tuning-v2"

headers = {
    "Authorization": github_token,
    "User-Agent": user_agent
}

response = requests.get(f"https://api.github.com/repos/{owner}/{repo}/readme", headers=headers)

readme_data = response.json()
encoded_content = readme_data.get('content', '')
decoded_content = base64.b64decode(encoded_content).decode('utf-8')

print(decoded_content)

# P-tuning v2


Source codes and data for
* [ACL 2022] [P-Tuning v2: Prompt Tuning Can Be Comparable to Finetuning Universally Across Scales and Tasks](https://arxiv.org/abs/2110.07602) 
* [⭐️ New: 2022.7] [Parameter-Efficient Prompt Tuning Makes Generalized and Calibrated Neural Text Retrievers](https://arxiv.org/pdf/2207.07087.pdf)  [[Code]](https://github.com/THUDM/P-tuning-v2/tree/main/PT-Retrieval)

An optimized prompt tuning strategy achieving comparable performance to fine-tuning on small/medium-sized models and sequence tagging challenges. 

Find our previous version [P-tuning v1](https://github.com/THUDM/P-tuning) for knowledge probing and few-shot SuperGLUE. Your kindly starring our repo can greatly encourage us to work harder :)

You may be also interested in our recent work [GLM-130B: An Open Bilingual Pre-trained Model (2022-10-06)](https://arxiv.org/abs/2210.02414). It is an open-sourced LLM outperforming GPT-3 175B over various benchmarks. Get model weights, do inference

能够发现，在调用request时，同样需要创建请求头（header），而和此前创建爬虫时的header不同的是，这里的header只需要输入Authorization（也就是Github token）和user-agent即可，其中user-agent和此前的爬虫中的user-agent一致。

&emsp;&emsp;为了方便后续调用，这里我们将上述过程封装为一个完整的函数，其中需要注意的是，上段代码中的owner和repo变量，我们统一以字典形式进行传入：

In [83]:
def get_github_readme(dic):
    
    github_token = os.getenv('GITHUB_TOKEN')
    user_agent = os.getenv('search_user_agent')
    
    owner = dic['owner']
    repo = dic['repo']

    headers = {
        "Authorization": github_token,
        "User-Agent": user_agent
    }

    response = requests.get(f"https://api.github.com/repos/{owner}/{repo}/readme", headers=headers)

    readme_data = response.json()
    encoded_content = readme_data.get('content', '')
    decoded_content = base64.b64decode(encoded_content).decode('utf-8')
    
    return decoded_content

In [84]:
dic = {'owner': 'THUDM', 'repo': 'P-tuning-v2'}

In [85]:
get_github_readme(dic)

"# P-tuning v2\n\n\nSource codes and data for\n* [ACL 2022] [P-Tuning v2: Prompt Tuning Can Be Comparable to Finetuning Universally Across Scales and Tasks](https://arxiv.org/abs/2110.07602) \n* [⭐️ New: 2022.7] [Parameter-Efficient Prompt Tuning Makes Generalized and Calibrated Neural Text Retrievers](https://arxiv.org/pdf/2207.07087.pdf)  [[Code]](https://github.com/THUDM/P-tuning-v2/tree/main/PT-Retrieval)\n\nAn optimized prompt tuning strategy achieving comparable performance to fine-tuning on small/medium-sized models and sequence tagging challenges. \n\nFind our previous version [P-tuning v1](https://github.com/THUDM/P-tuning) for knowledge probing and few-shot SuperGLUE. Your kindly starring our repo can greatly encourage us to work harder :)\n\nYou may be also interested in our recent work [GLM-130B: An Open Bilingual Pre-trained Model (2022-10-06)](https://arxiv.org/abs/2210.02414). It is an open-sourced LLM outperforming GPT-3 175B over various benchmarks. Get model weights, 

&emsp;&emsp;同时我们发现，GitHub上的项目主页网址的基本结构是`https://api.github.com/repos/{owner}/{repo}/readme `。其中owner就是项目的拥有者，而repo则是对应某个项目在Github上的名称。具体，如果我们希望在谷歌搜索获得的连接中进行逐个的readme获取，则首先需要筛选搜索结果中的项目主页链接，并且从中解析出每个连接的owner和repo：

In [86]:
search_results

[{'title': 'THUDM/P-tuning-v2: An optimized deep prompt tuning ... - GitHub',
  'link': 'https://github.com/THUDM/P-tuning-v2',
  'snippet': 'An optimized deep prompt tuning strategy comparable to fine-tuning across scales and tasks - GitHub - THUDM/P-tuning-v2: An optimized deep prompt tuning\xa0...'},
 {'title': 'THUDM/P-tuning: A novel method to tune language models ... - GitHub',
  'link': 'https://github.com/THUDM/P-tuning',
  'snippet': "[2021-10-15] P-tuning v2 is out! Check our Github repo. A novel method to tune language models. Codes and datasets for paper ``GPT understands, too''. Xiao\xa0..."},
 {'title': 'What parameters is really trained during P-tuning v2? · Issue #1 ...',
  'link': 'https://github.com/THUDM/P-tuning-v2/issues/1',
  'snippet': "Oct 15, 2021 ... Could you please clarify what parameters are really tuned during p-tuning? Is it MLP producing input embeddings for a large model? Or maybe it's\xa0..."},
 {'title': 'huggingface/peft: PEFT: State-of-the-art Param

In [87]:
def extract_github_repos(search_results):
    # 使用列表推导式筛选出项目主页链接
    repo_links = [result['link'] for result in search_results if '/issues/' not in result['link'] and '/blob/' not in result['link'] and 'github.com' in result['link'] and len(result['link'].split('/')) == 5]

    # 从筛选后的链接中提取owner和repo
    repos_info = [{'owner': link.split('/')[3], 'repo': link.split('/')[4]} for link in repo_links]

    return repos_info

repos_info = extract_github_repos(search_results)
print(repos_info)

[{'owner': 'THUDM', 'repo': 'P-tuning-v2'}, {'owner': 'THUDM', 'repo': 'P-tuning'}, {'owner': 'huggingface', 'repo': 'peft'}, {'owner': 'hiyouga', 'repo': 'ChatGLM-Efficient-Tuning'}]


而在得到了一组组owner和repo之后，即可循环调用get_github_readme函数获取这些项目readme文档了。

##### 1.4 基于Github的自动搜索问答机器人实现流程

&emsp;&emsp;在准备好了相关函数之后，接下来我们即可仿造知乎搜索问答机器人的代码流程，创建基于Github的自动搜索问答机器人了。这里我们需要类似的创建get_search_text_github函数、get_answer_github函数和auto_search_answer_github函数，分别用于获取单个链接的readme内容、获取多个链接的readme内容以及自动进行搜索和问答。三个函数定义过程如下：

In [106]:
def get_search_text_github(q, dic):
    
    title = dic['owner'] + '_' + dic['repo']
    title = windows_compatible_name(title)

    # 创建问题答案正文
    text = get_github_readme(dic)

    # 写入本地json文件
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")     
    json_data = [
        {
            "title": title,
            "content": text,
            "tokens": len(encoding.encode(text))
        }
    ]

    with open('./auto_search/%s/%s.json' % (q, title), 'w') as f:
        json.dump(json_data, f)

    return title

In [12]:
def get_answer_github(q):
    """
    当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 调用转化函数，将用户的问题转化为更适合在GitHub上搜索的关键词
    q = convert_keyword_github(q)
    
    # 默认搜索返回10个答案
    print('正在接入谷歌搜索，查找和问题相关的答案...')
    search_results = google_search(query=q, num_results=10, site_url='https://github.com/')
    results = extract_github_repos(search_results)
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    print('正在读取搜索的到的相关答案...')
    num_tokens = 0
    content = ''
    
    for dic in results:
        title = get_search_text_github(q, dic)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    print('正在进行最后的整理...')
    return(content)

In [108]:
def auto_search_answer_github(q):
    # 调用判别模型
    res = identify_model(q)
    if res == True:
        messages = [{"role": "user", "content": q}]
        res =run_conversation(messages=messages, 
                              functions_list=[get_answer_github], 
                              model="gpt-3.5-turbo-16k-0613", 
                              function_call={"name": "get_answer_github"})
    return(res)

然后即可测试函数效果：

In [113]:
auto_search_answer_github('请帮我介绍下P-tunning v2')

正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...


'P-tunning v2是一种优化的Prompt Tuning（P-tuning）策略，可以在小型/中型模型和序列标注任务上达到与Fine-tuning（微调）相当的性能。它是ACL 2022会议论文《P-Tuning v2: Prompt Tuning Can Be Comparable to Finetuning Universally Across Scales and Tasks》研究的一部分。\n\nP-tunning v2利用深度Prompt Tuning的思想，即为预训练的转换器的每个层输入应用连续的提示。深度Prompt Tuning增加了连续提示的容量，并在各种设置下缩小了与Fine-tuning之间的差距，特别是对于小型模型和困难任务。\n\n你可以在论文中找到更多关于P-tunning v2的详细信息以及其实验结果和性能指标。另外，论文中还提供了相关的代码和数据集，可以用于复现研究结果。'

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/b3bb70c1a14c20fa5797d6a5427a2f9.png" alt="b3bb70c1a14c20fa5797d6a5427a2f9" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/148b297c24e986c03ba86a101f66046.png" alt="148b297c24e986c03ba86a101f66046" style="zoom:33%;" />

In [114]:
auto_search_answer_github('请帮我详细介绍下P-tunning v2基本原理')

正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...


'P-Tuning v2是一种参数高效的微调方法，它允许有效地适应预训练的语言模型（PLMs）到各种下游应用，而不需要微调所有模型参数。微调大规模PLMs通常成本很高，因此P-Tuning v2只微调少量的（额外的）模型参数，极大减少了计算和存储成本。最新的P-Tuning v2技术可以达到与完整微调相当的性能。\n\nP-Tuning v2基本原理涉及以下步骤：\n\n1. **确定任务类型和模型架构**：首先，需要确定下游任务的类型（如序列到序列生成、文本分类、标记分类等）以及使用的基础语言模型的架构。P-Tuning v2可以适用于各种任务和模型架构。\n\n2. **添加适配器层**：将适配器层添加到原始语言模型中。适配器层是一个小型、单独训练的神经网络层，用于接收语言模型的隐藏表示，并输出符合特定任务的结果。适配器层与语言模型共享参数，只有少量的适配器参数需要训练。\n\n3. **选择适配器类型**：为适配器选择合适的类型。P-Tuning v2提供了多种适配器类型选择，如LoRA、Prefix Tuning、P-Tuning、Prompt Tuning和IA3。不同的适配器类型适用于不同的任务和场景，可以根据具体情况选择最合适的适配器类型。\n\n4. **训练适配器层**：使用下游任务的训练数据对适配器层进行训练。训练过程只涉及适配器层的参数，并且可以使用较小的学习率和较少的训练步骤来完成。通过仅训练适配器层，可以显著减少训练时间和计算资源消耗。\n\n5. **微调参数**：在完成适配器层训练后，可以选择对一些额外的模型参数进行微调。这些额外的参数可以是语言模型中的一些特定层或块，或者是以前的微调任务中的参数。这种微调是可选的，可以根据具体情况和资源约束进行决策。\n\n通过这些步骤，P-Tuning v2可以在减少计算和存储成本的同时，实现与完整微调相当的性能。这使得P-Tuning v2成为在计算资源有限的情况下，有效适应预训练模型到各种下游任务的一种有吸引力的选择。'

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/6ef368d5efe5a8574877026e001fc70.png" alt="6ef368d5efe5a8574877026e001fc70" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/05196730d8ab56b7d1c7a4af3758490.png" alt="05196730d8ab56b7d1c7a4af3758490" style="zoom:33%;" />

In [122]:
auto_search_answer_github('请帮我介绍下ChatGLM-6B')

正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...


'ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 General Language Model (GLM) 架构，具有 62 亿参数。模型通过大规模的中英双语标识符训练得到，结合了多种训练技术和优化方法。ChatGLM-6B 能够生成符合人类偏好的回答，并可用于各种对话任务，如问答、文本生成等。\n\nChatGLM-6B 的权重是开放的，你可以在 Hugging Face 模型仓库中找到它，并根据需要进行下载和使用。该模型也支持量化技术，在消费级显卡上进行本地部署。您可以使用提供的代码进行模型加载和生成对话。\n\n需要注意的是，虽然 ChatGLM-6B 在训练阶段遵循了合规性和准确性要求，但由于模型的规模较小且受到概率随机性的影响，无法保证生成内容的准确性。此外，模型还存在一些局限性，如容量较小、语言能力有限、易被误导等。在使用模型时，请遵循模型的许可协议和开源协议，并注意模型的局限性。如果有任何问题或反馈，欢迎与模型的开发团队联系。'

当然，除了这种单独问题单独搜索的方法外，还有一种直接调用get_answer_github函数获取相关文档，然后以system_message形式输入多轮对话函数chat_with_model中，以实现同一问题主题的多次反复提问：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/52ff93526ba637cac3eeb6049c0a406.png" alt="52ff93526ba637cac3eeb6049c0a406" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/0332b472a66d7b4c86398677afcee82.png" alt="0332b472a66d7b4c86398677afcee82" style="zoom:33%;" />

In [118]:
content = get_answer_github(q='请帮我介绍下LLAMA2')
content

正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...


'# Llama 2\n\nWe are unlocking the power of large language models. Our latest version of Llama is now accessible to individuals, creators, researchers and businesses of all sizes so that they can experiment, innovate and scale their ideas responsibly. \n\nThis release includes model weights and starting code for pretrained and fine-tuned Llama language models — ranging from 7B to 70B parameters.\n\nThis repository is intended as a minimal example to load [Llama 2](https://ai.meta.com/research/publications/llama-2-open-foundation-and-fine-tuned-chat-models/) models and run inference. For more detailed examples leveraging Hugging Face, see [llama-recipes](https://github.com/facebookresearch/llama-recipes/).\n\n## Updates post-launch\n\nSee [UPDATES.md](UPDATES.md).\n\n## Download\n\n⚠️ **7/18: We\'re aware of people encountering a number of download issues today. Anyone still encountering issues should remove all local files, re-clone the repository, and [request a new download link](htt

In [119]:
chat_with_model(functions_list=None, 
                prompt="请帮我介绍下LLAMA2", 
                model="gpt-3.5-turbo-16k-0613", 
                system_message=[{"role": "system", "content": content}])

模型回答: LLAMA2是一种深度学习语言模型，由Facebook的Meta AI团队开发。LLAMA代表"Language Learning with Adaptive Models"。LLAMA2基于GPT（Generative Pre-trained Transformer）架构，其目标是生成与人类文本相似的上下文相关文本。

LLAMA2的一个重要特点是其规模可扩展性，可以应用于许多不同的任务和数据集。它可以根据用户提供的文本提示生成自然、连贯的文本回复，可以用于对话系统、智能助手、文本生成等应用。

LLAMA2的训练过程需要大量的计算资源和数据来进行，但是训练好的模型可以在各种设备上进行推理和部署。Facebook的Meta AI团队已经释放了一系列预训练的LLAMA2模型，以供研究和实际应用使用。这些模型的规模从数百万到数十亿个参数不等，适用于不同规模的应用。

LLAMA2具有广泛的应用潜力，包括自动生成文本、自然语言处理任务、智能问答系统等。它是一个强大的语言生成工具，可以帮助改进人机交互体验、增强自然语言理解等应用。

需要注意的是，LLAMA2是一个非常复杂的模型，可能存在滥用的风险。用户和开发者在使用LLAMA2时需要遵循相应的使用准则和道德规范，确保安全和负责任的使用。


您还有其他问题吗？(输入退出以结束对话):  LLAMA2模型如何安装？


模型回答: 要安装LLAMA2模型，请按照以下步骤进行操作：

1. 首先，访问Meta AI网站并接受许可协议。您可以在[Llama 2下载页面](https://ai.meta.com/resources/models-and-libraries/llama-downloads/)找到下载链接。
2. 在下载页面上，选择您想要的LLAMA2模型（例如LLAMA2-7B或LLAMA2-13B）以及对应的权重文件和标记器。
3. 单击下载按钮，您将获得一个签名的URL链接。
4. 使用wget或浏览器等工具下载模型权重文件和标记器。
5. 下载完成后，您将得到一个包含LLAMA2权重文件和标记器的文件夹。

安装完成后，您可以使用LLAMA2模型进行推理和生成文本。


您还有其他问题吗？(输入退出以结束对话):  请给出一段LLAMA2模型运行的示例代码


模型回答: 当您获得LLAMA2模型的权重和标记器之后，您可以按照以下步骤安装并运行LLAMA2模型：

1. 克隆LLAMA2.c仓库：
```bash
git clone https://github.com/karpathy/llama2.c.git
```

2. 在克隆的仓库中运行以下命令以安装依赖项并构建运行代码：
```bash
cd llama2.c
pip install -e .
```

3. 下载LLAMA2模型权重和标记器，并将它们放入LLAMA2.c代码目录中。
   - 您可以从Meta AI网站下载LLAMA2模型权重和标记器，然后将它们放入`llama2.c`目录中。
   - 或者，您可以使用Hugging Face网站上的LLAMA2模型，下载权重并将其放置在`llama2.c`目录中，使用与预训练模型对应的标记器。

4. 运行LLAMA2模型的示例推理代码。例如，使用LLAMA2-7B模型，您可以运行以下命令：
```bash
torchrun --nproc_per_node 1 example_text_completion.py \
    --ckpt_dir llama-2-7b/ \
    --tokenizer_path tokenizer.model \
    --max_seq_len 128 --max_batch_size 4
```

根据您的LLAMA2模型类型和需求，您可能需要调整示例代码中的参数。确保提供正确的模型路径、标记器路径和所需的最大序列长度和批次大小等参数。

这样，您就可以使用LLAMA2模型执行文本完成任务。根据需要，您可以修改示例代码以适应其他LLAMA2模型的其他任务，如聊天模型等。


您还有其他问题吗？(输入退出以结束对话):  LLAMA2模型在本地部署时，有哪些硬件要求？


模型回答: LLAMA 2模型在本地部署时，具有以下硬件要求：

1. GPU：较大的LLAMA 2模型（如13B或70B）需要具备较高的显存容量。在使用这些模型时，您需要至少具备16GB或更大的显存。较小的LLAMA 2模型（如7B）对于显存要求较低，通常需要8GB或更大容量的显存。

2. CPU：如果您不使用GPU，可以在CPU上运行较小的LLAMA 2模型。LLAMA 2模型对于CPU的要求较低，但较大的模型可能需要一些计算资源和内存。

除了硬件要求之外，为了在本地成功部署LLAMA 2模型，您还需要下载模型权重和分词器，并正确设置运行环境（软件依赖项、Python环境等）。有关具体的安装和运行示例，请参考LLAMA 2模型的文档和示例代码。


您还有其他问题吗？(输入退出以结束对话):  退出


&emsp;&emsp;至此，我们即完整构建了Github搜索问答机器人。

#### 2.围绕Hugging face的智能搜索问答流程

&emsp;&emsp;接下来我们考虑将Hugging face接入GPT模型中。得益于Hugging face完善的API体系，相比前两个智能搜索问答项目，将Hugging face接入GPT模型的过程会简单很多，甚至都不用谷歌搜索API即可完成。总的来说，将Hugging face接入GPT模型的流程是，首先，借助此前定义的convert_keyword_github函数提取用户提问的模型关键词，然后借助Hugging face的Search API完成模型搜索，和Github类似，在Hugging face中，同一个模型会有多种类型的变种，这里我们可以搜索得到一些较为受欢迎的模型进行模型功能的查询。而具体的查询过程也同样是查询不同模型的readme文档，而Hugging face上的readme文档同样也是可以通过API快速获取。

&emsp;&emsp;这里我们首先尝试手动实现调用Hugging face API完成上述流程，然后再考虑将其封装在一个完整函数内。

##### 2.1 手动实现Hugging face API调用及信息获取

- 查询模型

&emsp;&emsp;这里我们可以通过如下过程，根据用户输入的模型关键词，查询与之相关的Hugging face上的一系列模型基本信息：

In [4]:
def search_models(query):
    url = "https://huggingface.co/api/models"
    params = {"search": query}
    response = requests.get(url, params=params)
    return response.json()

search_results = search_models("chatglm")

In [5]:
len(search_results)

229

作为模型社区，Hugging face上拥有海量模型，而光是与chatglm相关的模型，就有多大229个。这里我们根据模型的受欢迎程度（likes指标）对其进行降序排序，挑选较为受欢迎的模型进行问答：

In [6]:
sorted_results = sorted(search_results, key=lambda x: x['likes'], reverse=True)

In [7]:
sorted_results[:10]

[{'_id': '640f4f1409c94e1d9bca3ffc',
  'id': 'THUDM/chatglm-6b',
  'likes': 2594,
  'private': False,
  'downloads': 953259,
  'tags': ['pytorch',
   'chatglm',
   'custom_code',
   'zh',
   'en',
   'arxiv:2103.10360',
   'arxiv:2210.02414',
   'transformers',
   'glm',
   'thudm',
   'has_space',
   'region:us'],
  'modelId': 'THUDM/chatglm-6b'},
 {'_id': '64971933a9c42bc6a848d3f4',
  'id': 'THUDM/chatglm2-6b',
  'likes': 1598,
  'private': False,
  'downloads': 2589671,
  'tags': ['pytorch',
   'chatglm',
   'custom_code',
   'zh',
   'en',
   'arxiv:2103.10360',
   'arxiv:2210.02414',
   'arxiv:1911.02150',
   'transformers',
   'glm',
   'thudm',
   'has_space',
   'region:us'],
  'modelId': 'THUDM/chatglm2-6b'},
 {'_id': '6416f9b4ad63d650515a81f9',
  'id': 'THUDM/chatglm-6b-int4',
  'likes': 369,
  'private': False,
  'downloads': 16320,
  'tags': ['pytorch',
   'chatglm',
   'custom_code',
   'zh',
   'en',
   'transformers',
   'glm',
   'thudm',
   'has_space',
   'region:us']

In [8]:
sorted_results[3]['id']

'fb700/chatglm-fitness-RLHF'

- 获取模型readme文档

&emsp;&emsp;接下来即可根据模型ID，调用对应Hugging face API来查询各模型的说明文档。具体实现过程如下：

In [9]:
def get_model_readme(model_id):
    url = f"https://huggingface.co/{model_id}/resolve/main/README.md"
    response = requests.get(url)
    return response.text

In [10]:
model_readme = get_model_readme(sorted_results[3]['id'])

In [11]:
model_readme

'---\nlanguage:\n- zh\n- en\ntags:\n- chatglm-6b\n- chatglm2-6b\n- pytorch\n- peft\n- ft\n- sft\n- PPO\n- RLHF\n- RM\n- Transformer\nlicense: "apache-2.0"\n---\n- 模型体验地址：https://huggingface.co/spaces/fb700/chatglm-fitness-RLHF 测试用户过多，服务压力太大，为确保体验现设置密码，未来视情况定期更新密码，现账号密码为:test/qwer4321\n- 模型能力测试报告：https://huggingface.co/fb700/Bofan-chatglm-Best-lora/blob/main/modelapplytest.md\n# 重磅消息\n- 本项目经过多位网友实测，中文总结能力超越了GPT3.5各版本，健康咨询水平在同参数规模模型也出类拔萃，可能是任何个人和中小企业首选模型。\n# 重大突破\n- 虽然新版本的chatglm2-6b支持32k，但是我训练的模型在之前经优化，早就可以支持无限context，远大于4k、8K、16K......\n# ChatGLM-6B RLHF & LoRA Model\n\nChatGLM-6B 是开源中英双语对话模型，本次训练基于ChatGLM-6B 的第一代版本，在保留了初代模型对话流畅、部署门槛较低等众多优秀特性的基础之上开展训练。通过训练我们对模型有了更深刻的认知，LLM在一直在进化，好的方法和数据可以挖掘出模型的更大潜能。\n## 本次训练使用的方法\n\n- 首先，用40万条高质量数据进行强化训练，以提高模型的基础能力；\n- 第二，使用30万条人类反馈数据，构建一个表达方式规范优雅的语言模式（RM模型）；\n- 第三，在保留SFT阶段三分之一训练数据的同时，增加了30万条fitness数据，叠加RM模型，对ChatGLM-6B进行强化训练。\n- 成果，训练后在健康咨询，文档总结能力上不但强于chatglm-6b，而且部分能力上更是强于chatglm2-6b，配合“闻达”和“langchain-chatglm”等知识库项目，应用体验上对比chatglm-6b、chatglm2-6b和百川-7

- 基于模型readme文档的多轮对话

&emsp;&emsp;在获取了某个模型的readme文档之后，接下来即可将其作为system_message输入到chat_with_model多轮对话函数中进行多轮对话询问：

In [ ]:
chat_with_model(functions_list=None, 
                prompt="请帮我介绍下这个模型：%s" % sorted_results[3]['id'], 
                model="gpt-4-0613", 
                system_message=[{"role": "system", "content": model_readme}])

模型回答: `fb700/chatglm-fitness-RLHF` 是一个由用户[帛凡]在开源聊天机器人模型ChatGLM-6B基础上进行二次训练的模型。首先，原模型进行了40万条数据的强化训练以提高模型的基础能力，然后通过30万条人类反馈数据构建了一个表达方式规范优雅的语言模式。最后，又基于已有模型对ChatGLM-6B进行了强化训练，使其在健康咨询和文档总结能力上有显著提升。 

在部署和使用上，这个模型具有以下优点：
1. 兼容性：与ChatGLM-6B原模型的运行方式一致，用户可直接替换使用。
2. 性能：在采用FP16运行时，速度提升约20%。
3. 特性：该模型具有超强的自然对话理解力和 summarization 能力，理论上支持无限轮次的智能对话。

总的来说，在保留ChatGLM-6B原模型优点的基础上，通过二次训练，`fb700/chatglm-fitness-RLHF`在对话流畅度、主题理解、和回答问题等方面表现出更高的能力。


您还有其他问题吗？(输入退出以结束对话):  能介绍下chatglm在RLHF上训练的细节么？


模型回答: "ChatGLM-6B RLHF"模型是基于"ChatGLM-6B"模型进行了强化化训练的版本，其训练注入了大量的健康咨询类数据，并且使用了先进的训练策略来增强其对话流畅性、扩大应用领域、提高回答问题的准确度。

它的训练过程主要有以下三个步骤：

1. 首先，使用大约40万条高质量数据对模型进行强化训练，提升模型的基础能力。
2. 然后，使用约30万条人工反馈数据，构建了一个表达方式规范优雅的语言模式（RM模型）。
3. 最后，在保留SFT阶段三分之一训练数据的同时，增加了30万条健康咨询类数据，并叠加了RM模型，对ChatGLM-6B进行了强化训练。

"ChatGLM-6B RLHF"模型由于注入了大量的健康咨询类数据，并且通过先进的训练策略进行了训练，因此在诸如健康咨询、文档总结等领域的能力强于chatglm-6b，配合知识库项目，应用体验更好。同时，通过对原始模型进行了优化和处理，可以在原有的基础上提供更高的计算和存储效率。


In [65]:
def get_model_readme(model_id):
    url = f"https://huggingface.co/{model_id}/resolve/main/README.md"
    response = requests.get(url)
    return response.text

In [66]:
model_readme = get_model_readme("THUDM/chatglm-6b-int4")

In [67]:
model_readme

'---\nlanguage:\n- zh\n- en\ntags:\n- glm\n- chatglm\n- thudm\n---\n# ChatGLM-6B-INT4\n<p align="center">\n    👋 Join our <a href="https://join.slack.com/t/chatglm/shared_invite/zt-1udqapmrr-ocT1DS_mxWe6dDY8ahRWzg" target="_blank">Slack</a> and <a href="https://github.com/THUDM/ChatGLM-6B/blob/main/resources/WECHAT.md" target="_blank">WeChat</a>\n</p>\n\n## 介绍\nChatGLM-6B 是一个开源的、支持中英双语问答的对话语言模型，基于 [General Language Model (GLM)](https://github.com/THUDM/GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。ChatGLM-6B 使用了和 [ChatGLM](https://chatglm.cn) 相同的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。\n\nChatGLM-6B-INT4 是 ChatGLM-6B 量化后的模型权重。具体的，ChatGLM-6B-INT4 对 ChatGLM-6B 中的 28 个 GLM Block 进行了 INT4 量化，没有对 Embedding 和 LM Head 进行量化。量化后的模型理论上 6G 显存（使用 CPU 即内存）即可推理，具有在嵌入式设备（如树莓派）上运行的可能。\n\n在 CPU 上运行时，会根据硬件自动编译 CPU Kernel ，请确保已安装 GCC 和 OpenMP （Linux一般已安装，对于Windows则需手动安装），以获得最佳并行计算能力。\n\n## 软件依赖\n\n```shell\npip install prot

In [69]:
chat_with_model(functions_list=None, 
                    prompt="请帮我介绍下ChatGLM-6B-INT4模型", 
                    model="gpt-4-0613", 
                    system_message=[{"role": "system", "content": model_readme}])

模型回答: ChatGLM-6B-INT4 是基于 General Language Model（GLM）架构的开源对话语言模型，支持中英双语并具有62亿参数。通过模型量化技术，此模型可以在消费级的显卡上本地部署。在 INT4 量化级别下，理论上最低只需要6GB显存即可运行。

ChatGLM-6B-INT4 是 ChatGLM-6B 进行量化后的模型权重。具体来说，ChatGLM-6B-INT4 对 ChatGLM-6B 中的28个 GLM Block 进行了 INT4 量化，但并没有对 Embedding 和 LM Head 进行量化。

此模型经过约1T标识符的中英双语训练，且通过了监督微调、反馈自助、人类反馈强化学习等技术优化，能够生成符合人类偏好的回答。量化后的模型也具有在嵌入式设备（例如树莓派）上运行的可能性。


您还有其他问题吗？(输入退出以结束对话):  ChatGLM-6B-INT4模型如何在本地进行安装部署呢？


模型回答: 在您的本地环境部署 ChatGLM-6B-INT4 模型，您可以遵循以下步骤：

首先，您需要确保您的环境已经安装了Python，并能联网下载依赖和模型。

1. 安装必要的Python库。在命令行中输入下列命令：

```shell
pip install protobuf transformers==4.27.1 cpm_kernels
```
注意: 上面的命令需要使用pip命令来从Python的包管理器PyPI安装所需的库。如果您的系统上有多个Python版本，可能需要用pip3代替pip。

2. 导入必要的模型和库, 通过以下Python代码进行调用：

```python
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
```
注意：以上代码将模型加载到显存中，如果你的显卡显存不够，你也可以选择使用模型量化以节省显存，或者在 CPU 上运行。

关于更多的使用说明，包括如何运行命令行和网页版本的 DEMO，以及使用模型量化以节省显存，请参考他们的[Github Repo](https://github.com/THUDM/ChatGLM-6B)。


您还有其他问题吗？(输入退出以结束对话):  ChatGLM-6B-INT4模型可以在CPU上进行安装么？


模型回答: 当然可以。实际上，ChatGLM-6B-INT4模型在cpu和嵌入式设备上运行时，会根据硬件自动编译CPU Kernel。确保已安装GCC和OpenMP，以获取最佳并行计算能力。

ChatGLM-6B-INT4是ChatGLM-6B量化后的模型权重，它对ChatGLM-6B中的28个GLM Block进行了INT4量化，没有对Embedding和LM Head进行量化。量化后的模型理论上6G显存即可推理。

在软件依赖方面，请先安装以下所需的包：

```shell
pip install protobuf transformers==4.27.1 cpm_kernels
```

这样就可以在本地环境中安装和使用ChatGLM-6B-INT4模型了，安装步骤也十分简单和直观。


您还有其他问题吗？(输入退出以结束对话):  能否提供ChatGLM-6B-INT4模型运行示例代码呢？


模型回答: 以下是使用ChatGLM-6B模型的示例代码：

```python
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response)
```

此代码首先实例化模型并加载预训练权重，然后可能与模型进行一次对话。

注意：在实际运行时，请务必确保您已正确安装所有必要的依赖项，并拥有适当的硬件要求。


您还有其他问题吗？(输入退出以结束对话):  退出


##### 2.2 全自动Hugging face模型问答机器人

In [17]:
sorted_results[:5][0]['id']

'THUDM/chatglm-6b'

In [23]:
def chat_with_huggingface(q):

    # 调用转化函数，将用户的问题转化为模型关键词
    query = convert_keyword_github(q)
    
    search_results = search_models(query)
    sorted_results = sorted(search_results, key=lambda x: x['likes'], reverse=True)[:5]
    
    print('现有5个与“%s”相关的模型如下：' % query)
    for dic in sorted_results:
        print(dic['id'])


    model_id = input('请输入想查询的模型ID：')

    model_readme = get_model_readme(model_id)
    chat_with_model(functions_list=None, 
                    prompt=q, 
                    model="gpt-4-0613", 
                    system_message=[{"role": "system", "content": model_readme}])
    
    res = input('是否还需要查询其他相关模型，输入“yes”继续查询其他模型，回复“no”则退出本次对话')
    if res == 'yes':
        chat_with_huggingface(q)

In [21]:
q = '请帮我介绍下DeepSpeedChat'
chat_with_huggingface(q)

现有5个与“DeepSpeedChat”相关的模型如下：
zen-E/deepspeed-chat-step1-model-opt1.3b
zen-E/deepspeed-chat-step2-model-opt350m
zen-E/deepspeed-chat-step3-rlhf-actor-model-opt1.3b
YzZ-George/DeepSpeed-Chat-OPT-1.3B-3-3-3datasets
YzZ-George/DeepSpeed-Chat-OPT-1.3B-4-4-1datasets


请输入想查询的模型ID： zen-E/deepspeed-chat-step1-model-opt1.3b


模型回答: DeepSpeedChat是一个由Microsoft的DeepSpeed团队开发的科研项目，主要用于训练大型对话模型。这个项目使得在资源有限的硬件环境下，可以训练出类似于GPT-3这样的大型语言模型。

DeepSpeedChat使用了一种叫做“Zero Redundancy Optimizer”（简称ZERO）的技术，在训练过程中将模型参数放在GPU之间进行分散存储，以减少内存的使用量。它还使用超大批量训练和模型并行化技术，用于高效地训练大型模型。

DeepSpeedChat还提供了一种名为“OneBitAdam”的通信库，用以改善参数更新的通信效率。此外，DeepSpeedChat还配备了一套模型压缩工具，可以有效地减少模型的存储需求。

总的来说，DeepSpeedChat是一个高效、易用且功能强大的工具集，为训练大型对话模型提供了便利。


您还有其他问题吗？(输入退出以结束对话):  ZERO技术是什么呢？


模型回答: ZERO技术是一个由微软DeepSpeed团队开发的用于优化深度学习模型训练的技术。ZERO全称为ZeRO，即“零冗余优化器”。它的主要目标是减少GPU训练中的冗余，从而让模型训练更加高效。

在传统的深度学习模型训练中，每个GPU都需要保存一份完整的模型参数和优化器状态，这在处理大型模型时会导致GPU内存资源的大量消耗。ZERO技术通过在所有GPU之间划分并共享这些参数和状态，显著减少了每个GPU所需的内存，使得我们可以在相同的硬件上训练更大的模型，或者用更小的硬件来训练同样大的模型。

ZERO技术主要包括三个阶段：ZeRO-1、ZeRO-2和ZeRO-3，每个阶段都进一步减少了GPU内存的需求。例如，ZeRO-3阶段能够实现模型参数、优化器状态和梯度的全部分布式存储，从而使得GPU内存需求几乎与模型大小无关。

总的来说，ZERO技术通过减少冗余和更高效的利用资源，大大提高了深度学习模型训练的效率。


您还有其他问题吗？(输入退出以结束对话):  退出
是否还需要查询其他相关模型，输入“yes”继续查询其他模型，回复“no”则退出本次对话 yes


现有5个与“DeepSpeedChat”相关的模型如下：
zen-E/deepspeed-chat-step1-model-opt1.3b
zen-E/deepspeed-chat-step2-model-opt350m
zen-E/deepspeed-chat-step3-rlhf-actor-model-opt1.3b
YzZ-George/DeepSpeed-Chat-OPT-1.3B-3-3-3datasets
YzZ-George/DeepSpeed-Chat-OPT-1.3B-4-4-1datasets


请输入想查询的模型ID： zen-E/deepspeed-chat-step2-model-opt350m


模型回答: DeepSpeedChat是一个通过DeepSpeedExamples提供的应用程序，它使用DeepSpeed进行大规模模型的训练。DeepSpeed是一种深度学习优化库，可以进行大规模的模型训练和推理。主要通过模型并行性、优化的激活和参数检查点、混合精度训练等技术，提高系统效率以及模型规模。

在DeepSpeedChat中，开发人员可以训练自己的聊天机器人模型，例如OPT系列模型，在多个数据集上进行优化。这种训练方式使得模型可以根据特定的聊天对话来生成或预测回复，为聊天机器人或对话系统的开发提供了强大的支持。


您还有其他问题吗？(输入退出以结束对话):  退出


&emsp;&emsp;至此，我们就完整介绍了如何借助搜索引擎API和GPT模型的Function calling功能来实时拓展GPT模型知识库的方法。本节结束后，需要按照要求将相关代码统一保存至autoSearch.py文件中，方便后续进行调用。需要注意的是，这部分内容在后续介绍长文本匹配时将会有更进一步的进阶使用方法介绍，届时我们将搭配Embedding和微调技术，来让模型更好的围绕更长的文本进行知识提取和吸收。